In [ ]:
"""
╔══════════════════════════════════════════════════════════════════════════════╗
║                  AIVI DATA INTEGRATION - BLOCO 1 v4.1                        ║
║              CONFIGURAÇÃO COMPLETA DO AMBIENTE + UX/UI + LOGS                ║
╠══════════════════════════════════════════════════════════════════════════════╣
║  Versão: 4.1 - Consolidado, Modular, Inteligente                            ║
║  Data: 2025-01-15                                                            ║
║  Prompt Base: AIVI v4.1 (com UX/UI, Schema Logging, Exploração)             ║
╠══════════════════════════════════════════════════════════════════════════════╣
║  CONTEÚDO DESTE BLOCO:                                                       ║
║    1. Imports e Constantes AIVI 2025                                         ║
║    2. Sistema UX/UI com Timer (PARTE 0.10)                                   ║
║    3. Log de Campos + Exploração (PARTE 12.3 + 14)                           ║
║    4. Regex Patterns para Detecção de Conteúdo                               ║
║    5. Validações Não-Destrutivas                                             ║
║    6. FileManager Expandido                                                  ║
║    7. Inicialização e Teste                                                  ║
╠══════════════════════════════════════════════════════════════════════════════╣
║  PROTOCOLO BLOCO-A-BLOCO:                                                    ║
║    ✅ Execute este bloco                                                     ║
║    ✅ Valide todos os outputs                                                ║
║    ✅ Teste a GUI (janela de seleção de pasta)                               ║
║    ✅ Confirme: "BLOCO 1 OK" ou reporte problemas                            ║
║    ⏭️  Só então prosseguiremos para BLOCO 2                                  ║
╚══════════════════════════════════════════════════════════════════════════════╝
"""

# ═══════════════════════════════════════════════════════════════════════════
# SEÇÃO 1: IMPORTS E CONFIGURAÇÕES BÁSICAS
# ═══════════════════════════════════════════════════════════════════════════

print("╔" + "═" * 78 + "╗")
print("║" + " 🚀 BLOCO 1: CONFIGURAÇÃO COMPLETA DO AMBIENTE".center(78) + "║")
print("╚" + "═" * 78 + "╝")
print()

# Imports padrão
import sys
import os
import json
import logging
import warnings
from pathlib import Path
from datetime import datetime
from typing import Dict, List, Tuple, Optional, Any
import re
import unicodedata

# Imports para análise de dados
import pandas as pd
import numpy as np

# Imports para GUI
import tkinter as tk
from tkinter import filedialog, messagebox

# Configurações
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: f'{x:,.4f}')

print("📦 SEÇÃO 1.1: Imports")
print("-" * 80)
print("   ✅ Bibliotecas padrão: sys, os, json, logging, datetime")
print("   ✅ Análise de dados: pandas, numpy")
print("   ✅ GUI: tkinter")
print("   ✅ Configurações aplicadas")
print()

# Verificar versões
modulos_carregados = {
    'Python': sys.version.split()[0],
    'Pandas': pd.__version__,
    'NumPy': np.__version__
}

print("📋 Versões:")
for modulo, versao in modulos_carregados.items():
    print(f"   • {modulo}: {versao}")
print()

# ═══════════════════════════════════════════════════════════════════════════
# SEÇÃO 2: CONSTANTES AIVI 2025
# ═══════════════════════════════════════════════════════════════════════════

print("📊 SEÇÃO 1.2: Constantes AIVI 2025")
print("-" * 80)

# Constantes principais
CONSTANTES_AIVI = {
    'ANO_VIGENCIA': 2025,
    'VERSAO_SISTEMA': '4.1',
    'DATA_ATUALIZACAO': '2025-01-15',

    # Parâmetros estatísticos
    'PERCENTIL_INFERIOR': 10,
    'PERCENTIL_SUPERIOR': 90,
    'DESVIOS_PADRAO_BATENTE': 2.5,

    # Pesos AIVI
    'PESO_CONFORMIDADE': 0.50,
    'PESO_SEVERIDADE': 0.30,
    'PESO_FREQUENCIA': 0.20,

    # Limites de alerta
    'LIMITE_CRITICO_PCT': 30.0,
    'LIMITE_ALERTA_PCT': 10.0,

    # Configurações de análise
    'MESES_MINIMO_HISTORICO': 6,
    'MESES_IDEAL_HISTORICO': 12,
    'CONFIANCA_MINIMA_MAPEAMENTO': 0.80
}

print(f"   ✅ Ano de vigência: {CONSTANTES_AIVI['ANO_VIGENCIA']}")
print(f"   ✅ Versão do sistema: {CONSTANTES_AIVI['VERSAO_SISTEMA']}")
print(f"   ✅ {len(CONSTANTES_AIVI)} constantes definidas")
print()

# Schema AIVI - Campos obrigatórios
SCHEMA_AIVI_CAMPOS_OBRIGATORIOS = [
    'Período',
    'Centro',
    'Sigla',
    'Cód Grupo de produto',
    'Expedição c/ Veículo',
    'Variação Interna',
    '% VI',
    'LI Atual (%)',
    'LS Atual (%)'
]

print(f"   ✅ Schema AIVI: {len(SCHEMA_AIVI_CAMPOS_OBRIGATORIOS)} campos obrigatórios")
print()

# Dicionário de sinônimos expandido
DICIONARIO_SINONIMOS = {
    'Centro': [
        'Centro', 'Código de Centro', 'Cod Centro', 'CódigoCentro',
        'Centro Operacional', 'ID Centro', 'Centro_ID', 'CENTRO',
        'Cod_Centro', 'Centro_Operacional', 'Plant', 'Planta'
    ],

    'Sigla': [
        'Sigla', 'Sigla de Centro', 'Sigla Centro', 'Sigla Base',
        'Base', 'Sigla_Centro', 'SIGLA', 'SiglaBase', 'Sigla_Base',
        'Sigla do Centro', 'Nome Base', 'Base Operacional'
    ],

    'Cód Grupo de produto': [
        'Cód Grupo de produto', 'Código Produto', 'Cod Produto',
        'Grupo Produto', 'CodGrupoProduto', 'Cód_Grupo_produto',
        'PRODUTO', 'Produto_Cod', 'CodGrupo', 'Material Group',
        'Grupo de Material', 'Product Group'
    ],

    'Expedição c/ Veículo': [
        'Expedição c/ Veículo', 'Expedição', 'Expedicao',
        'Volume Expedido', 'Exped_Veiculo', 'EXPEDICAO', 'Exped',
        'Expedicao_Veiculo', 'Volume_Expedido', 'Dispatch Volume',
        'Volume Carregado', 'Carregamento'
    ],

    'Variação Interna': [
        'Variação Interna', 'Variacao Interna', 'VI', 'Var Interna',
        'VarInt', 'VARIACAO', 'Var_Interna', 'VariacaoInterna',
        'Internal Variation', 'Variation'
    ],

    '% VI': [
        '% VI', 'Percentual VI', 'VI %', 'Percentual Variacao Interna',
        'Perc VI', 'VI_Percent', 'Variation %'
    ],

    'LI Atual (%)': [
        'LI Atual (%)', 'Limite Inferior', 'LI', 'Lim Inferior',
        'Limite_Inferior', 'LIMITE_INF', 'LimInf', 'Lim_Inferior',
        'Lower Limit', 'LL'
    ],

    'LS Atual (%)': [
        'LS Atual (%)', 'Limite Superior', 'LS', 'Lim Superior',
        'Limite_Superior', 'LIMITE_SUP', 'LimSup', 'Lim_Superior',
        'Upper Limit', 'UL'
    ],

    'Período': [
        'Período', 'Periodo', 'Data', 'Data Referencia',
        'Mês/Ano', 'Mes Ano', 'Competencia', 'Period', 'Date'
    ],

    'Mês do exercício': [
        'Mês do exercício', 'Mês', 'Mes', 'Mês Exercício',
        'MesExercicio', 'MES', 'Mes_Exercicio', 'Mes_Ref', 'Month'
    ],

    'Ano do documento do material': [
        'Ano do documento do material', 'Ano', 'Ano Documento',
        'AnoDoc', 'ANO', 'Ano_Documento', 'Ano_Ref', 'Year'
    ]
}

print(f"   ✅ Dicionário de sinônimos: {len(DICIONARIO_SINONIMOS)} campos principais")
print(f"   ✅ Total de variações mapeadas: {sum(len(v) for v in DICIONARIO_SINONIMOS.values())}")
print()

# ═══════════════════════════════════════════════════════════════════════════
# SEÇÃO 3: REGEX PATTERNS PARA DETECÇÃO DE CONTEÚDO
# ═══════════════════════════════════════════════════════════════════════════

print("🔍 SEÇÃO 1.3: Regex Patterns para Detecção de Conteúdo")
print("-" * 80)

PATTERNS_CONTEUDO = {
    # CENTRO: 4 dígitos numéricos como string
    'centro': {
        'pattern': r'^\d{4}$',
        'description': 'Centro: 4 dígitos',
        'exemplo': '1001'
    },

    # CÓDIGO DE MATERIAL/PRODUTO: formatos xx.xxx.xxx, x.xxx.xxx, xxxxxxxx, xxxxxxx
    'codigo_material': {
        'pattern': r'^(\d{1,2}\.\d{3}\.\d{3}|\d{7,8})$',
        'description': 'Código Material: xx.xxx.xxx ou 7-8 dígitos',
        'exemplo': '10.123.456'
    },

    # CÓDIGO DE GRUPO: pode ter texto com underscore ou números
    'codigo_grupo': {
        'pattern': r'^([A-Z0-9_]+|\d{1,2}\.\d{3}\.\d{3}|\d{7,8})$',
        'description': 'Código Grupo: TEXTO_TEXTO ou numérico',
        'exemplo': 'DIESEL_S10'
    },

    # TRANSPORTE/DOCUMENTO: 10 dígitos
    'documento_transporte': {
        'pattern': r'^\d{10}$',
        'description': 'Documento Transporte: 10 dígitos',
        'exemplo': '1234567890'
    },

    # PERÍODO/DATA: diversos formatos
    'data': {
        'pattern': r'^\d{2}/\d{2}/\d{4}$|^\d{4}-\d{2}-\d{2}$|^\d{1,2}\.\d{4}$|^\d{2}\.\d{4}$',
        'description': 'Data: dd/mm/yyyy, yyyy-mm-dd ou mm.yyyy',
        'exemplo': '01/01/2025'
    },

    # VALORES NUMÉRICOS: com separadores regionais
    'valor_numerico': {
        'pattern': r'^-?\d{1,3}(\.\d{3})*(,\d+)?$|^-?\d+(,\d{3})*(\.\d+)?$|^-?\d+(\.\d+)?$',
        'description': 'Valor: 1.234,56 ou 1,234.56 ou 1234.56',
        'exemplo': '1.234,56'
    },

    # PERCENTUAL
    'percentual': {
        'pattern': r'^-?\d+([.,]\d+)?%?$',
        'description': 'Percentual: 12,34% ou 12.34',
        'exemplo': '12,34%'
    },

    # SIGLA BASE: 2-4 letras maiúsculas
    'sigla_base': {
        'pattern': r'^[A-Z]{2,4}$',
        'description': 'Sigla: 2-4 letras maiúsculas',
        'exemplo': 'RLAM'
    },

    # TEXTO GERAL
    'texto': {
        'pattern': r'^[A-Za-zÀ-ÿ\s\-_/]+$',
        'description': 'Texto: letras, espaços, hífen',
        'exemplo': 'Gasolina Comum'
    }
}

print(f"   ✅ {len(PATTERNS_CONTEUDO)} patterns definidos")
for tipo, config in list(PATTERNS_CONTEUDO.items())[:3]:
    print(f"      • {tipo}: {config['description']}")
print(f"      • ... e mais {len(PATTERNS_CONTEUDO) - 3} patterns")
print()

# ═══════════════════════════════════════════════════════════════════════════
# SEÇÃO 4: FUNÇÕES AUXILIARES - DETECÇÃO E VALIDAÇÃO
# ═══════════════════════════════════════════════════════════════════════════

print("⚙️  SEÇÃO 1.4: Funções Auxiliares")
print("-" * 80)

def normalizar_string(s: str) -> str:
    """
    Normaliza string para comparação (remove acentos, lowercase, apenas alfanuméricos).

    Args:
        s: String a normalizar

    Returns:
        String normalizada
    """
    s = unicodedata.normalize('NFKD', str(s))
    s = s.encode('ASCII', 'ignore').decode('ASCII')
    s = s.lower()
    s = re.sub(r'[^a-z0-9]', '', s)
    return s

def calcular_similaridade(s1: str, s2: str) -> float:
    """
    Calcula similaridade entre duas strings (0.0 a 1.0).

    Args:
        s1: Primeira string
        s2: Segunda string

    Returns:
        Score de similaridade
    """
    from difflib import SequenceMatcher
    s1_norm = normalizar_string(s1)
    s2_norm = normalizar_string(s2)
    return SequenceMatcher(None, s1_norm, s2_norm).ratio()

def detectar_tipo_conteudo(valor: str, verbose: bool = False) -> Dict[str, Any]:
    """
    Detecta tipo de conteúdo usando regex patterns.

    Args:
        valor: String a analisar
        verbose: Se True, mostra detalhes

    Returns:
        Dict com tipo detectado e confiança
    """
    if pd.isna(valor) or str(valor).strip() == '':
        return {'tipo': 'VAZIO', 'confianca': 1.0, 'pattern': None}

    valor_str = str(valor).strip()
    matches = []

    # Testar todos os patterns
    for tipo, config in PATTERNS_CONTEUDO.items():
        if re.match(config['pattern'], valor_str):
            matches.append(tipo)
            if verbose:
                print(f"   ✅ Match: {tipo} - {config['description']}")

    if not matches:
        return {'tipo': 'DESCONHECIDO', 'confianca': 0.0, 'pattern': None}

    # Priorizar mais específico
    prioridade = ['centro', 'codigo_material', 'documento_transporte', 'data',
                  'percentual', 'valor_numerico', 'sigla_base', 'codigo_grupo', 'texto']

    for tipo_prior in prioridade:
        if tipo_prior in matches:
            return {
                'tipo': tipo_prior.upper(),
                'confianca': 1.0 if len(matches) == 1 else 0.8,
                'pattern': PATTERNS_CONTEUDO[tipo_prior]['pattern'],
                'alternativas': [m for m in matches if m != tipo_prior]
            }

    return {'tipo': matches[0].upper(), 'confianca': 0.7, 'pattern': PATTERNS_CONTEUDO[matches[0]]['pattern']}

def analisar_coluna_completa(serie: pd.Series, amostra: int = 100) -> Dict:
    """
    Analisa coluna completa para determinar tipo predominante.

    Args:
        serie: Série pandas
        amostra: Número de valores a analisar

    Returns:
        Dict com análise completa
    """
    valores_nao_nulos = serie.dropna().astype(str).head(amostra)

    if len(valores_nao_nulos) == 0:
        return {
            'tipo_predominante': 'VAZIO',
            'confianca': 0.0,
            'tipos_encontrados': {},
            'exemplos': []
        }

    tipos_contagem = {}
    exemplos_por_tipo = {}

    for valor in valores_nao_nulos:
        deteccao = detectar_tipo_conteudo(valor)
        tipo = deteccao['tipo']

        tipos_contagem[tipo] = tipos_contagem.get(tipo, 0) + 1

        if tipo not in exemplos_por_tipo:
            exemplos_por_tipo[tipo] = []
        if len(exemplos_por_tipo[tipo]) < 3:
            exemplos_por_tipo[tipo].append(valor)

    tipo_predominante = max(tipos_contagem, key=tipos_contagem.get)
    total = len(valores_nao_nulos)
    confianca = tipos_contagem[tipo_predominante] / total

    return {
        'tipo_predominante': tipo_predominante,
        'confianca': confianca,
        'tipos_encontrados': tipos_contagem,
        'exemplos': exemplos_por_tipo.get(tipo_predominante, [])[:3],
        'total_analisado': total
    }

def identificar_linha_cabecalho(df: pd.DataFrame, dicionario_termos: List[str]) -> int:
    """
    Identifica linha de cabeçalho usando dicionário de termos conhecidos.

    Args:
        df: DataFrame bruto
        dicionario_termos: Lista de termos que aparecem em cabeçalhos

    Returns:
        Índice da linha de cabeçalho (ou -1 se não encontrado)
    """
    termos_norm = [normalizar_string(t) for t in dicionario_termos]

    for idx in range(min(20, len(df))):
        row = df.iloc[idx]
        valores_norm = [normalizar_string(str(v)) for v in row if pd.notna(v)]

        matches = sum(1 for termo in termos_norm if any(termo in v for v in valores_norm))
        pct_match = matches / len(termos_norm) if termos_norm else 0

        if pct_match >= 0.5:
            return idx

    return -1

def validar_transformacao(df_antes: pd.DataFrame, df_depois: pd.DataFrame,
                         nome_transformacao: str, permitir_reducao: bool = False) -> Dict:
    """
    Valida transformação comparando estados antes/depois.

    Args:
        df_antes: DataFrame original
        df_depois: DataFrame transformado
        nome_transformacao: Nome da operação
        permitir_reducao: Se True, permite redução de linhas/colunas

    Returns:
        Dict com resultado da validação
    """
    resultado = {
        'nome': nome_transformacao,
        'passou': True,
        'alertas': [],
        'metricas': {}
    }

    # Linhas
    linhas_antes = len(df_antes)
    linhas_depois = len(df_depois)
    diff_linhas = linhas_depois - linhas_antes

    resultado['metricas']['linhas'] = {
        'antes': linhas_antes,
        'depois': linhas_depois,
        'diferenca': diff_linhas
    }

    if diff_linhas < 0 and not permitir_reducao:
        resultado['passou'] = False
        resultado['alertas'].append(f"❌ PERDA DE LINHAS: {abs(diff_linhas):,}")

    # Colunas
    colunas_antes = len(df_antes.columns)
    colunas_depois = len(df_depois.columns)

    resultado['metricas']['colunas'] = {
        'antes': colunas_antes,
        'depois': colunas_depois,
        'diferenca': colunas_depois - colunas_antes
    }

    # NULLs
    nulls_antes = df_antes.isnull().sum().sum()
    nulls_depois = df_depois.isnull().sum().sum()

    resultado['metricas']['nulls'] = {
        'antes': int(nulls_antes),
        'depois': int(nulls_depois),
        'diferenca': int(nulls_depois - nulls_antes)
    }

    return resultado

def formatar_tabela_resumo(dados: Dict[str, Any], titulo: str = "RESUMO") -> None:
    """
    Formata dicionário como tabela imprimível.

    Args:
        dados: Dicionário com dados
        titulo: Título da tabela
    """
    print(f"\n╔{'═' * 78}╗")
    print(f"║ {titulo.center(76)} ║")
    print(f"╠{'═' * 78}╣")

    for chave, valor in dados.items():
        chave_fmt = chave.replace('_', ' ').title().ljust(35)

        if isinstance(valor, (int, np.integer)):
            valor_fmt = f"{valor:,}".rjust(40)
        elif isinstance(valor, (float, np.floating)):
            valor_fmt = f"{valor:,.2f}".rjust(40)
        elif isinstance(valor, bool):
            valor_fmt = ("✅ SIM" if valor else "❌ NÃO").rjust(40)
        else:
            valor_str = str(valor)
            if len(valor_str) > 40:
                valor_fmt = (valor_str[:37] + "...").rjust(40)
            else:
                valor_fmt = valor_str.rjust(40)

        print(f"║ {chave_fmt} : {valor_fmt} ║")

    print(f"╚{'═' * 78}╝")

print("   ✅ normalizar_string()")
print("   ✅ calcular_similaridade()")
print("   ✅ detectar_tipo_conteudo()")
print("   ✅ analisar_coluna_completa()")
print("   ✅ identificar_linha_cabecalho()")
print("   ✅ validar_transformacao()")
print("   ✅ formatar_tabela_resumo()")
print()

# ═══════════════════════════════════════════════════════════════════════════
# SEÇÃO 5: SISTEMA UX/UI COM TIMER (PARTE 0.10)
# ═══════════════════════════════════════════════════════════════════════════

print("🎨 SEÇÃO 1.5: Sistema UX/UI com Timer e Memória")
print("-" * 80)

class DialogoPadraoAIVI:
    """
    Classe base para diálogos interativos com:
    - Memória de última escolha
    - Timer de 10s com countdown visual
    - Botões padronizados
    - Logging transparente
    """

    CONFIG_DIR = Path.home() / '.aivi_config'
    TIMER_PADRAO = 10

    def __init__(self, tipo: str, titulo: str, mensagem: str):
        self.tipo = tipo
        self.titulo = titulo
        self.mensagem = mensagem
        self.CONFIG_DIR.mkdir(exist_ok=True)
        self.config_file = self.CONFIG_DIR / f'aivi_{tipo}_last.json'

    def carregar_ultima_escolha(self) -> Optional[str]:
        """Carrega última escolha salva."""
        if self.config_file.exists():
            try:
                with open(self.config_file, 'r', encoding='utf-8') as f:
                    config = json.load(f)
                ultima = config.get('last_choice')

                # Validar se é caminho e se existe
                if self.tipo in ['diretorio', 'arquivo']:
                    if ultima and Path(ultima).exists():
                        return ultima
                    return None
                return ultima
            except:
                return None
        return None

    def salvar_escolha(self, escolha: str) -> None:
        """Salva escolha atual."""
        config = {
            'last_choice': str(escolha),
            'timestamp': datetime.now().isoformat(),
            'tipo': self.tipo
        }
        with open(self.config_file, 'w', encoding='utf-8') as f:
            json.dump(config, f, indent=2, ensure_ascii=False)

def selecionar_diretorio_trabalho() -> Path:
    """
    Seleciona diretório de trabalho com UX/UI padrão:
    - Memória de última pasta
    - Timer de 10s
    - Botões padronizados

    Returns:
        Path do diretório selecionado
    """
    dialogo = DialogoPadraoAIVI(
        'diretorio_trabalho',
        'Diretório de Trabalho AIVI',
        'Selecione a pasta onde os dados e outputs serão salvos'
    )

    ultimo_dir = dialogo.carregar_ultima_escolha()

    # Criar janela
    root = tk.Tk()
    root.withdraw()  # Ocultar janela principal

    # Se tem última pasta, criar janela com timer
    if ultimo_dir:
        root.deiconify()
        root.title("AIVI - Diretório de Trabalho")
        root.geometry("600x300")

        # Centralizar
        x = (root.winfo_screenwidth() // 2) - 300
        y = (root.winfo_screenheight() // 2) - 150
        root.geometry(f"+{x}+{y}")

        resultado = {'path': None, 'cancelado': False, 'timeout': False}
        contador = [10]

        # Frame
        frame = tk.Frame(root, padx=20, pady=20, bg='white')
        frame.pack(fill=tk.BOTH, expand=True)

        # Título
        tk.Label(
            frame,
            text="Diretório de Trabalho AIVI",
            font=('Arial', 14, 'bold'),
            bg='white'
        ).pack(pady=(0, 15))

        # Mensagem
        tk.Label(
            frame,
            text=f"📂 Última pasta usada:\n{ultimo_dir}",
            font=('Arial', 9),
            bg='#E8F5E9',
            fg='#2E7D32',
            padx=10,
            pady=10,
            wraplength=550
        ).pack(pady=(0, 10))

        # Timer
        label_timer = tk.Label(
            frame,
            text=f"{contador[0]}s",
            font=('Arial', 20, 'bold'),
            fg='#FF4444',
            bg='white'
        )
        label_timer.pack(pady=(5, 20))

        def countdown():
            if contador[0] > 0 and not resultado['cancelado']:
                contador[0] -= 1
                label_timer.config(text=f"{contador[0]}s")
                root.after(1000, countdown)
            elif contador[0] == 0 and not resultado['cancelado']:
                resultado['timeout'] = True
                root.quit()
                root.destroy()

        # Botões
        frame_btns = tk.Frame(frame, bg='white')
        frame_btns.pack(side=tk.BOTTOM, pady=20)

        def escolher_nova():
            resultado['cancelado'] = True
            root.withdraw()
            diretorio = filedialog.askdirectory(
                title="Diretório de Trabalho AIVI",
                initialdir=ultimo_dir
            )
            resultado['path'] = diretorio if diretorio else ultimo_dir
            root.quit()
            root.destroy()

        def usar_ultima():
            resultado['cancelado'] = True
            resultado['path'] = ultimo_dir
            root.quit()
            root.destroy()

        tk.Button(
            frame_btns,
            text="Escolher Nova Pasta",
            command=escolher_nova,
            width=22,
            height=2,
            font=('Arial', 10, 'bold'),
            bg='#4CAF50',
            fg='white',
            cursor='hand2'
        ).pack(side=tk.LEFT, padx=10)

        tk.Button(
            frame_btns,
            text="Usar Última Pasta",
            command=usar_ultima,
            width=22,
            height=2,
            font=('Arial', 10),
            bg='#2196F3',
            fg='white',
            cursor='hand2'
        ).pack(side=tk.LEFT, padx=10)

        # Iniciar timer
        root.after(1000, countdown)
        root.mainloop()

        # Processar resultado
        if resultado.get('timeout'):
            print(f"   ⏱️  Timeout (10s) - usando última pasta")
            resultado['path'] = ultimo_dir

        if resultado['path']:
            dialogo.salvar_escolha(resultado['path'])
            return Path(resultado['path'])

    # Sem última pasta ou escolher nova
    root.withdraw()
    diretorio = filedialog.askdirectory(title="Diretório de Trabalho AIVI")

    if diretorio:
        dialogo.salvar_escolha(diretorio)
        print(f"   ✅ Diretório selecionado: {diretorio}")
        return Path(diretorio)
    else:
        raise ValueError("❌ Nenhum diretório selecionado")

print("   ✅ DialogoPadraoAIVI (classe base)")
print("   ✅ selecionar_diretorio_trabalho() (com timer + memória)")
print()

# ═══════════════════════════════════════════════════════════════════════════
# SEÇÃO 6: FILEMANAGER EXPANDIDO COM LOGS DE CAMPOS
# ═══════════════════════════════════════════════════════════════════════════

print("📁 SEÇÃO 1.6: FileManager Expandido")
print("-" * 80)

class FileManagerAIVI:
    """
    Gerenciador de arquivos e logs AIVI v4.1.

    Funcionalidades:
    - Estrutura de pastas completa
    - Log de execução (geral + específico)
    - Log de campos separado (PARTE 12.3)
    - Pasta de exploração (PARTE 14)
    """

    def __init__(self, diretorio_base: Optional[Path] = None):
        """Inicializa FileManager."""

        self.timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        print(f"   🕐 Timestamp: {self.timestamp}")

        # Selecionar diretório base
        if diretorio_base is None:
            self.diretorio_base = selecionar_diretorio_trabalho()
        else:
            self.diretorio_base = Path(diretorio_base)

        print(f"   📂 Diretório base: {self.diretorio_base}")

        # Criar estrutura
        self.diretorio_execucao = self.diretorio_base / f"AIVI_DataIntegration_{self.timestamp}"
        self._criar_estrutura()
        self._configurar_logging()

        # Log inicial
        self.logger.info("═" * 80)
        self.logger.info("NOVA EXECUÇÃO - AIVI DATA INTEGRATION v4.1")
        self.logger.info("═" * 80)
        self.logger.info(f"Timestamp: {self.timestamp}")
        self.logger.info(f"Diretório: {self.diretorio_execucao}")
        self.logger.info(f"Python: {sys.version.split()[0]}")
        self.logger.info(f"Pandas: {pd.__version__}")

        print()
        print(f"   ✅ FileManager inicializado")
        print(f"   ✅ Sistema de logs ativo")
        print()

    def _criar_estrutura(self):
        """Cria estrutura de pastas."""

        # Estrutura expandida v4.1
        self.diretorios = {
            'logs': self.diretorio_execucao / '01_Logs',
            'logs_campos': self.diretorio_execucao / '01_Logs' / 'logs_campos',  # NOVO
            'logs_execucao': self.diretorio_execucao / '01_Logs' / 'logs_execucao',  # NOVO
            'dados_entrada': self.diretorio_execucao / '02_Dados_Entrada',
            'dados_processados': self.diretorio_execucao / '03_Dados_Processados',
            'dados_integrados': self.diretorio_execucao / '04_Dados_Integrados',
            'exploracao': self.diretorio_execucao / '05_Exploracao',  # NOVO (PARTE 14)
            'relatorios': self.diretorio_execucao / '06_Relatorios',
            'validacoes': self.diretorio_execucao / '07_Validacoes',
            'exports': self.diretorio_execucao / '08_Exports'
        }

        # Criar todas as pastas
        for nome, caminho in self.diretorios.items():
            caminho.mkdir(parents=True, exist_ok=True)

    def _configurar_logging(self):
        """Configura sistema de logging."""

        log_file = self.diretorios['logs_execucao'] / f'aivi_analise_{self.timestamp}.log'

        # Configurar logger
        self.logger = logging.getLogger(f'AIVI_{self.timestamp}')
        self.logger.setLevel(logging.DEBUG)

        # Handler para arquivo
        fh = logging.FileHandler(log_file, encoding='utf-8')
        fh.setLevel(logging.DEBUG)

        # Handler para console
        ch = logging.StreamHandler()
        ch.setLevel(logging.INFO)

        # Formato
        formatter = logging.Formatter(
            '%(asctime)s | %(levelname)-8s | %(message)s',
            datefmt='%Y-%m-%d %H:%M:%S'
        )
        fh.setFormatter(formatter)
        ch.setFormatter(formatter)

        self.logger.addHandler(fh)
        self.logger.addHandler(ch)

    def salvar_metadata(self, metadata_adicional: Dict = None) -> Path:
        """Salva metadata da execução."""

        metadata = {
            'timestamp': self.timestamp,
            'versao': CONSTANTES_AIVI['VERSAO_SISTEMA'],
            'data_atualizacao': CONSTANTES_AIVI['DATA_ATUALIZACAO'],
            'diretorio_base': str(self.diretorio_base),
            'diretorio_execucao': str(self.diretorio_execucao),
            'estrutura_pastas': {k: str(v) for k, v in self.diretorios.items()},
            'python_version': sys.version,
            'pandas_version': pd.__version__
        }

        if metadata_adicional:
            metadata.update(metadata_adicional)

        metadata_file = self.diretorios['logs'] / 'metadata.json'
        with open(metadata_file, 'w', encoding='utf-8') as f:
            json.dump(metadata, f, indent=2, ensure_ascii=False)

        return metadata_file

    def registrar_schema_campos(self, df: pd.DataFrame, fonte: str, arquivo_origem: str) -> Path:
        """
        Registra schema completo de campos em log separado (PARTE 12.3).

        Args:
            df: DataFrame a registrar
            fonte: Nome da fonte (ex: 'SAP_YSMM_VI_ACOMP')
            arquivo_origem: Nome do arquivo original

        Returns:
            Path do arquivo de log de campos
        """
        schema_info = {
            'fonte': fonte,
            'arquivo_origem': arquivo_origem,
            'timestamp': datetime.now().isoformat(),
            'total_registros': len(df),
            'total_colunas': len(df.columns),
            'campos': {}
        }

        # Analisar cada campo
        for i, col in enumerate(df.columns, 1):
            serie = df[col]

            campo_info = {
                'posicao': i,
                'nome_original': col,
                'tipo_pandas': str(serie.dtype),
                'count_total': len(serie),
                'count_nulos': int(serie.isnull().sum()),
                'pct_nulos': float(serie.isnull().sum() / len(serie) * 100) if len(serie) > 0 else 0,
                'count_unicos': int(serie.nunique()),
                'pct_cardinalidade': float(serie.nunique() / len(serie) * 100) if len(serie) > 0 else 0
            }

            # Análise de conteúdo
            analise = analisar_coluna_completa(serie, amostra=50)
            campo_info['tipo_conteudo_detectado'] = analise['tipo_predominante']
            campo_info['confianca_deteccao'] = analise['confianca']
            campo_info['exemplos'] = analise['exemplos']

            schema_info['campos'][col] = campo_info

        # Salvar
        log_campos_file = self.diretorios['logs_campos'] / f'{fonte}_{self.timestamp}_schema.json'
        with open(log_campos_file, 'w', encoding='utf-8') as f:
            json.dump(schema_info, f, indent=2, ensure_ascii=False)

        self.logger.info(f"Schema de campos registrado: {log_campos_file.name}")

        return log_campos_file

print("   ✅ FileManagerAIVI (classe expandida)")
print("   ✅ Suporte a logs de campos (PARTE 12.3)")
print("   ✅ Pasta de exploração (PARTE 14)")
print()

# ═══════════════════════════════════════════════════════════════════════════
# SEÇÃO 7: INICIALIZAÇÃO E TESTE
# ═══════════════════════════════════════════════════════════════════════════

print("🚀 SEÇÃO 1.7: Inicializando FileManager")
print("-" * 80)
print()

# Inicializar (vai abrir GUI)
try:
    fm = FileManagerAIVI()

    # Criar aliases para compatibilidade
    gerenciador = fm
    gp = fm

    # Aliases de diretórios
    DIR_LOGS = fm.diretorios['logs']
    DIR_LOGS_CAMPOS = fm.diretorios['logs_campos']
    DIR_LOGS_EXECUCAO = fm.diretorios['logs_execucao']
    DIR_DADOS_ENTRADA = fm.diretorios['dados_entrada']
    DIR_DADOS_PROCESSADOS = fm.diretorios['dados_processados']
    DIR_DADOS_INTEGRADOS = fm.diretorios['dados_integrados']
    DIR_EXPLORACAO = fm.diretorios['exploracao']
    DIR_RELATORIOS = fm.diretorios['relatorios']
    DIR_VALIDACOES = fm.diretorios['validacoes']
    DIR_EXPORTS = fm.diretorios['exports']

    # Salvar metadata inicial
    metadata_file = fm.salvar_metadata({
        'bloco': 1,
        'fase': 'configuracao_completa',
        'modulos_carregados': modulos_carregados,
        'constantes': CONSTANTES_AIVI,
        'patterns_definidos': len(PATTERNS_CONTEUDO),
        'sinonimos_mapeados': sum(len(v) for v in DICIONARIO_SINONIMOS.values())
    })

    print()

except Exception as e:
    print()
    print("╔" + "═" * 78 + "╗")
    print("║" + " ❌ ERRO NA INICIALIZAÇÃO ".center(78) + "║")
    print("╚" + "═" * 78 + "╝")
    print()
    print(f"Erro: {str(e)}")
    import traceback
    print(traceback.format_exc())
    raise

# ═══════════════════════════════════════════════════════════════════════════
# RESULTADO FINAL DO BLOCO 1
# ═══════════════════════════════════════════════════════════════════════════

print("╔" + "═" * 78 + "╗")
print("║" + " ✅ BLOCO 1 CONCLUÍDO COM SUCESSO ".center(78) + "║")
print("╚" + "═" * 78 + "╝")
print()

# Resumo em tabela formatada
formatar_tabela_resumo({
    'timestamp': fm.timestamp,
    'versao_sistema': CONSTANTES_AIVI['VERSAO_SISTEMA'],
    'diretorio_execucao': fm.diretorio_execucao.name,
    'total_pastas_criadas': len(fm.diretorios),
    'constantes_aivi': len(CONSTANTES_AIVI),
    'campos_obrigatorios': len(SCHEMA_AIVI_CAMPOS_OBRIGATORIOS),
    'patterns_regex': len(PATTERNS_CONTEUDO),
    'sinonimos_mapeados': sum(len(v) for v in DICIONARIO_SINONIMOS.values()),
    'funcoes_auxiliares': 7,
    'sistema_ux_ui': 'ATIVO (timer + memória)',
    'log_campos': 'ATIVO (PARTE 12.3)',
    'exploracao': 'ATIVO (PARTE 14)'
}, "RESUMO DO BLOCO 1")

print()
print("📂 ESTRUTURA DE PASTAS CRIADA:")
print("-" * 80)
for nome, caminho in fm.diretorios.items():
    print(f"   {nome.ljust(20)} → {caminho.name}")

print()
print("🔗 VARIÁVEIS GLOBAIS DISPONÍVEIS:")
print("-" * 80)
print("   • fm / gerenciador / gp (FileManager)")
print("   • DIR_LOGS, DIR_LOGS_CAMPOS, DIR_DADOS_ENTRADA, etc")
print("   • CONSTANTES_AIVI (dict)")
print("   • SCHEMA_AIVI_CAMPOS_OBRIGATORIOS (list)")
print("   • DICIONARIO_SINONIMOS (dict)")
print("   • PATTERNS_CONTEUDO (dict)")
print("   • Funções: normalizar_string(), detectar_tipo_conteudo(), etc")

print()
print("📋 PRÓXIMOS PASSOS:")
print("-" * 80)
print("   1. ✅ Verifique se o diretório foi criado corretamente")
print("   2. ✅ Confirme que todas as pastas existem (9 pastas)")
print("   3. ✅ Abra o arquivo de log em 01_Logs/logs_execucao/")
print("   4. 📤 Envie: 'BLOCO 1 OK' para prosseguir")
print("   5. ⏭️  BLOCO 2 será o próximo (Carregador de Arquivos)")

print()
print("🔗 CAMINHO COMPLETO:")
print(f"   {fm.diretorio_execucao}")
print()
print("═" * 80)

╔══════════════════════════════════════════════════════════════════════════════╗
║                 🚀 BLOCO 1: CONFIGURAÇÃO COMPLETA DO AMBIENTE                 ║
╚══════════════════════════════════════════════════════════════════════════════╝

📦 SEÇÃO 1.1: Imports
--------------------------------------------------------------------------------
   ✅ Bibliotecas padrão: sys, os, json, logging, datetime
   ✅ Análise de dados: pandas, numpy
   ✅ GUI: tkinter
   ✅ Configurações aplicadas

📋 Versões:
   • Python: 3.11.9
   • Pandas: 2.3.3
   • NumPy: 2.3.3

📊 SEÇÃO 1.2: Constantes AIVI 2025
--------------------------------------------------------------------------------
   ✅ Ano de vigência: 2025
   ✅ Versão do sistema: 4.1
   ✅ 14 constantes definidas

   ✅ Schema AIVI: 9 campos obrigatórios

   ✅ Dicionário de sinônimos: 11 campos principais
   ✅ Total de variações mapeadas: 111

🔍 SEÇÃO 1.3: Regex Patterns para Detecção de Conteúdo
---------------------------------------------------------

In [18]:
"""
═══════════════════════════════════════════════════════════════════════════════
AIVI DATA INTEGRATION - BLOCO 2: CARREGADOR MODULAR - ARQUIVO 1
═══════════════════════════════════════════════════════════════════════════════
Versão: 3.0 - Modular (um arquivo por vez)
Arquivo: YSMM_VI_ACOMP (SAP - Base de Dados Históricos)

FUNCIONALIDADES:
  ✅ Seleção via GUI
  ✅ Suporte robusto a .XLS e .XLSX
  ✅ Limpeza SAP automática (remove col A, linhas 1,2,3,5)
  ✅ Detecção automática de schema (Parte 16)
  ✅ Listagem COMPLETA de todas as colunas
  ✅ Validação de campos obrigatórios
  ✅ Backup automático
  ✅ Logging completo

APÓS ESTE BLOCO:
  • df_sap disponível globalmente
  • Aguarda feedback do usuário
  • Próximo: BLOCO 3 (Arquivo 2)
═══════════════════════════════════════════════════════════════════════════════
"""

import pandas as pd
from pathlib import Path
import tkinter as tk
from tkinter import filedialog, messagebox
from difflib import SequenceMatcher

print("╔" + "═" * 78 + "╗")
print("║" + " BLOCO 2: CARREGADOR MODULAR - ARQUIVO 1/N ".center(78) + "║")
print("╚" + "═" * 78 + "╝")
print()

# ══════════════════════════════════════════════════════════════════════════════
# FUNÇÃO AUXILIAR: Similaridade de Strings
# ══════════════════════════════════════════════════════════════════════════════

def calcular_similaridade(s1, s2):
    """
    Calcula similaridade entre duas strings usando SequenceMatcher.

    Args:
        s1: String 1
        s2: String 2

    Returns:
        float: Similaridade entre 0.0 e 1.0 (0% a 100%)
    """
    return SequenceMatcher(None, s1.lower(), s2.lower()).ratio()


# ══════════════════════════════════════════════════════════════════════════════
# CLASSE: CarregadorArquivo (MODULAR - reutilizável)
# ══════════════════════════════════════════════════════════════════════════════

class CarregadorArquivo:
    """
    Carregador modular de arquivos Excel com detecção automática de schema.

    Design Pattern: Template Method
    - Métodos genéricos para qualquer arquivo
    - Métodos específicos por tipo de arquivo (SAP, ESO, etc)
    """

    def __init__(self, gerenciador):
        self.gerenciador = gerenciador
        self.logger = gerenciador.logger
        self.df = None
        self.arquivo_original = None
        self.tipo_arquivo = None
        self.mapeamento_colunas = {}
        self.log_deteccao = []

    def selecionar_arquivo_gui(self, titulo, mensagem):
        """Abre GUI para seleção de arquivo."""
        root = tk.Tk()
        root.withdraw()
        root.lift()
        root.attributes('-topmost', True)
        root.attributes('-alpha', 0.0)
        root.update()

        messagebox.showinfo(titulo, mensagem.strip())

        arquivo = filedialog.askopenfilename(
            title=titulo,
            filetypes=[
                ("Excel files", "*.xlsx *.xls"),
                ("All files", "*.*")
            ]
        )

        root.destroy()

        if not arquivo:
            raise ValueError(f"❌ Nenhum arquivo selecionado")

        return Path(arquivo)

    def carregar_excel_robusto(self, caminho):
        """
        Carrega Excel com múltiplos engines (tolerante a falhas).

        Ordem de tentativa:
          1. Engine específico da extensão (openpyxl ou xlrd)
          2. Engine automático (pandas decide)
        """
        extensao = caminho.suffix.lower()

        print(f"📖 Carregando arquivo Excel ({extensao})...")

        # Definir engines
        if extensao == '.xlsx':
            engines = ['openpyxl', None]
        elif extensao == '.xls':
            engines = ['xlrd', None]
        else:
            engines = [None, 'openpyxl', 'xlrd']

        # Tentar cada engine
        ultimo_erro = None

        for i, engine in enumerate(engines, 1):
            try:
                if engine:
                    print(f"   Tentativa {i}/{len(engines)}: engine='{engine}'")
                    df = pd.read_excel(caminho, header=None, engine=engine)
                else:
                    print(f"   Tentativa {i}/{len(engines)}: engine automático")
                    df = pd.read_excel(caminho, header=None)

                print(f"   ✅ Sucesso! {df.shape[0]:,} linhas × {df.shape[1]} colunas")
                self.logger.info(f"Carregado: {caminho.name} ({df.shape[0]} × {df.shape[1]})")
                return df

            except Exception as e:
                ultimo_erro = e
                print(f"   ❌ Falhou: {str(e)[:60]}...")
                continue

        # Nenhum engine funcionou
        print()
        print("❌ ERRO: Não foi possível ler o arquivo")

        if extensao == '.xls':
            print()
            print("💡 SOLUÇÃO para .XLS:")
            print("   1. Instale xlrd: pip install xlrd")
            print("   2. OU converta para .XLSX no Excel")

        raise Exception(f"Falha ao ler: {ultimo_erro}")

    def limpar_sap_bruto(self, df_bruto):
        """
        Limpeza específica para arquivos SAP YSMM_VI_ACOMP.

        Transformações:
          1. Remove coluna A (índice 0) - coluna vazia do SAP
          2. Remove linhas 1, 2, 3, 5 - cabeçalhos e vazias
          3. Linha 4 vira cabeçalho
        """
        print()
        print("🧹 Aplicando limpeza SAP (YSMM_VI_ACOMP)...")
        print("-" * 80)

        print(f"   📊 Original: {df_bruto.shape[0]:,} linhas × {df_bruto.shape[1]} colunas")

        df = df_bruto.copy()

        # PASSO 1: Remover primeira coluna
        if df.shape[1] > 0:
            df = df.iloc[:, 1:]
            print(f"   ✅ Removida coluna A (índice 0)")

        # PASSO 2: Processar cabeçalho
        if len(df) > 4:
            # Linha 4 (índice 3) vira cabeçalho
            novo_cabecalho = df.iloc[3].values

            # Remover linhas 0,1,2,4 e pegar de 5 em diante
            df = df.iloc[5:].reset_index(drop=True)
            df.columns = novo_cabecalho

            print(f"   ✅ Removidas linhas 1, 2, 3, 5")
            print(f"   ✅ Cabeçalho definido (linha 4 original)")

        print(f"   📊 Limpo: {df.shape[0]:,} linhas × {df.shape[1]} colunas")
        print()

        self.logger.info(f"Limpeza SAP aplicada: {df.shape[0]} × {df.shape[1]}")

        return df

    def detectar_colunas_automatico(self, df, campos_esperados):
        """
        Detecção automática de colunas usando similaridade de strings.

        Implementa Parte 16 do prompt: Inteligência de Dados.

        Args:
            df: DataFrame com colunas a mapear
            campos_esperados: dict {campo_padrao: [sinonimos]}

        Returns:
            (df_mapeado, mapeamento, log_deteccao, sucesso)
        """
        print("🔍 DETECÇÃO AUTOMÁTICA DE COLUNAS")
        print("-" * 80)

        mapeamento = {}
        log_deteccao = []
        campos_nao_encontrados = []

        # Para cada campo esperado
        for campo_padrao, sinonimos in campos_esperados.items():
            melhor_score = 0.0
            melhor_match = None
            melhor_coluna_df = None

            # Procurar nas colunas do DataFrame
            for col_df in df.columns:
                col_df_str = str(col_df).strip()

                # Calcular similaridade com cada sinônimo
                for sinonimo in sinonimos:
                    # Exata?
                    if col_df_str == sinonimo:
                        score = 1.0
                    else:
                        # Fuzzy match
                        score = calcular_similaridade(col_df_str, sinonimo)

                    if score > melhor_score:
                        melhor_score = score
                        melhor_match = sinonimo
                        melhor_coluna_df = col_df

            # Threshold: 70%
            if melhor_score >= 0.70:
                # Mapear se nome diferente
                if melhor_coluna_df != campo_padrao:
                    mapeamento[melhor_coluna_df] = campo_padrao

                # Status visual
                if melhor_score >= 0.95:
                    status = "✅ AUTO"
                    auto = True
                elif melhor_score >= 0.80:
                    status = "⚠️  REVISAR"
                    auto = False
                else:
                    status = "⚠️  BAIXA"
                    auto = False

                print(f"   {status} '{melhor_coluna_df}' → '{campo_padrao}'")
                print(f"           Confiança: {melhor_score:.1%} (match: '{melhor_match}')")

                log_deteccao.append({
                    'coluna_original': melhor_coluna_df,
                    'campo_padrao': campo_padrao,
                    'confianca': melhor_score,
                    'match_com': melhor_match,
                    'automatico': auto
                })
            else:
                print(f"   ❌ '{campo_padrao}' - Não encontrado (melhor: {melhor_score:.1%})")
                campos_nao_encontrados.append(campo_padrao)

                log_deteccao.append({
                    'coluna_original': None,
                    'campo_padrao': campo_padrao,
                    'confianca': melhor_score,
                    'automatico': False,
                    'status': 'NAO_ENCONTRADO'
                })

        print()

        # Aplicar mapeamento
        if mapeamento:
            df = df.rename(columns=mapeamento)
            print(f"   🔄 {len(mapeamento)} colunas renomeadas")
            self.logger.info(f"Mapeamento: {len(mapeamento)} colunas")

        print()

        # Resultado
        sucesso = len(campos_nao_encontrados) == 0

        if campos_nao_encontrados:
            print(f"⚠️  {len(campos_nao_encontrados)} campos não encontrados:")
            for campo in campos_nao_encontrados:
                print(f"      • {campo}")
            print()
        else:
            print("✅ Todos os campos obrigatórios mapeados!")
            print()

        return df, mapeamento, log_deteccao, sucesso

    def listar_todas_colunas(self, df):
        """
        Lista TODAS as colunas com informações detalhadas.

        Para cada coluna mostra:
          - Tipo de dados
          - Quantidade de nulos
          - Quantidade de valores únicos
        """
        print()
        print("📑 LISTAGEM COMPLETA DE COLUNAS")
        print("═" * 80)
        print(f"📏 Total: {len(df.columns)} colunas")
        print()

        for i, col in enumerate(df.columns, 1):
            tipo = df[col].dtype
            nulos = df[col].isna().sum()
            pct_nulos = (nulos / len(df) * 100) if len(df) > 0 else 0
            unicos = df[col].nunique()

            print(f"   {i:2d}. {col}")
            print(f"       ├─ Tipo: {tipo}")
            print(f"       ├─ Nulos: {nulos:,} ({pct_nulos:.1f}%)")
            print(f"       └─ Únicos: {unicos:,}")

        print()

    def mostrar_preview_dados(self, df, n_linhas=3):
        """Mostra preview das primeiras linhas."""
        print("📊 PREVIEW DOS DADOS (primeiras 5 colunas)")
        print("-" * 80)

        colunas_preview = list(df.columns[:5])
        print(df[colunas_preview].head(n_linhas).to_string(index=False))
        print()

    def salvar_backup(self, df, nome_base):
        """Salva backup na pasta de entrada."""
        timestamp = self.gerenciador.timestamp

        arquivo_destino = self.gerenciador.diretorios['dados_entrada'] / \
                         f"{nome_base}_{timestamp}.xlsx"

        try:
            df.to_excel(arquivo_destino, index=False, engine='openpyxl')
            self.logger.info(f"Backup: {arquivo_destino.name}")
            print(f"   ✅ Backup: {arquivo_destino.name}")
            return arquivo_destino
        except Exception as e:
            print(f"   ⚠️  Erro ao salvar backup: {str(e)}")
            self.logger.error(f"Erro backup: {str(e)}")
            return None

    def salvar_log_deteccao(self, nome_arquivo="deteccao_schema"):
        """Salva log de detecção em JSON."""
        if not self.log_deteccao:
            return None

        timestamp = self.gerenciador.timestamp
        log_file = self.gerenciador.diretorios['logs'] / \
                  f"{nome_arquivo}_{timestamp}.json"

        import json
        with open(log_file, 'w', encoding='utf-8') as f:
            json.dump(self.log_deteccao, f, indent=2, ensure_ascii=False)

        self.logger.info(f"Log detecção: {log_file.name}")
        return log_file

    def carregar_ysmm_vi_acomp(self):
        """
        Pipeline completo para carregar YSMM_VI_ACOMP (SAP).

        Etapas:
          1. Seleção via GUI
          2. Carregamento robusto
          3. Limpeza SAP
          4. Detecção automática de schema
          5. Listagem de todas as colunas
          6. Preview dos dados
          7. Backup
        """
        print("╔" + "═" * 78 + "╗")
        print("║" + " ARQUIVO 1: YSMM_VI_ACOMP (SAP - BASE HISTÓRICA) ".center(78) + "║")
        print("╚" + "═" * 78 + "╝")
        print()

        mensagem = """
╔═══════════════════════════════════════════════════╗
║  📂 ARQUIVO SAP - YSMM_VI_ACOMP                   ║
╠═══════════════════════════════════════════════════╣
║                                                   ║
║  Transação: YSMM_VI_ACOMP                         ║
║  Conteúdo: Dados históricos de variações internas║
║  Período: Recomendado 12+ meses                   ║
║                                                   ║
║  ⚠️  IMPORTANTE:                                  ║
║                                                   ║
║  • Selecione arquivo ORIGINAL (BRUTO) do SAP      ║
║  • NÃO faça limpeza manual                        ║
║  • Aceita .XLS ou .XLSX                           ║
║  • Nome típico: 2025-2024-YSMM_VI_ACOMP.xlsx      ║
║                                                   ║
╚═══════════════════════════════════════════════════╝
        """

        try:
            # ═══════════════════════════════════════════════════════════════
            # ETAPA 1: Seleção
            # ═══════════════════════════════════════════════════════════════

            self.arquivo_original = self.selecionar_arquivo_gui(
                titulo="[1/N] YSMM_VI_ACOMP (SAP - BRUTO)",
                mensagem=mensagem
            )

            self.tipo_arquivo = "SAP_YSMM_VI_ACOMP"

            print(f"✅ Arquivo selecionado:")
            print(f"   📁 Nome: {self.arquivo_original.name}")
            print(f"   📊 Tamanho: {self.arquivo_original.stat().st_size:,} bytes")
            print(f"   🔧 Tipo: {self.arquivo_original.suffix}")
            print()

            # ═══════════════════════════════════════════════════════════════
            # ETAPA 2: Carregamento
            # ═══════════════════════════════════════════════════════════════

            df_bruto = self.carregar_excel_robusto(self.arquivo_original)

            # ═══════════════════════════════════════════════════════════════
            # ETAPA 3: Limpeza SAP
            # ═══════════════════════════════════════════════════════════════

            df_limpo = self.limpar_sap_bruto(df_bruto)

            # ═══════════════════════════════════════════════════════════════
            # ETAPA 4: Detecção automática de schema
            # ═══════════════════════════════════════════════════════════════

            # Campos obrigatórios SAP
            campos_sap = {
                'Centro': ['Centro', 'Código de Centro', 'Cod Centro'],
                'Cód Grupo de produto': ['Cód Grupo de produto', 'Cod Grupo de produto'],
                'Expedição': ['Expedição c/ Veí', 'Expedição c/ Veículo', 'Expedição'],
                'Variação Interna': ['Variação Interna', 'Variacao Interna', 'VI'],
                'Mês': ['Mês do exercício', 'Mes do exercicio', 'Mês'],
                'Ano': ['Ano do documento do material', 'Ano do documento', 'Ano']
            }

            df_mapeado, mapeamento, log, sucesso = \
                self.detectar_colunas_automatico(df_limpo, campos_sap)

            self.df = df_mapeado
            self.mapeamento_colunas = mapeamento
            self.log_deteccao = log

            # ═══════════════════════════════════════════════════════════════
            # ETAPA 5: Listar TODAS as colunas
            # ═══════════════════════════════════════════════════════════════

            self.listar_todas_colunas(self.df)

            # ═══════════════════════════════════════════════════════════════
            # ETAPA 6: Preview
            # ═══════════════════════════════════════════════════════════════

            self.mostrar_preview_dados(self.df)

            # ═══════════════════════════════════════════════════════════════
            # ETAPA 7: Backup
            # ═══════════════════════════════════════════════════════════════

            print("💾 Salvando backup...")
            backup = self.salvar_backup(self.df, "SAP_YSMM_Limpo")

            log_file = self.salvar_log_deteccao("deteccao_sap")
            if log_file:
                print(f"   ✅ Log detecção: {log_file.name}")

            print()

            # ═══════════════════════════════════════════════════════════════
            # RESULTADO FINAL
            # ═══════════════════════════════════════════════════════════════

            print("╔" + "═" * 78 + "╗")
            print("║" + " ✅ ARQUIVO SAP CARREGADO COM SUCESSO ".center(78) + "║")
            print("╚" + "═" * 78 + "╝")
            print()

            print("📊 RESUMO DO CARREGAMENTO:")
            print("-" * 80)
            print(f"   • Arquivo: {self.arquivo_original.name}")
            print(f"   • Registros: {len(self.df):,}")
            print(f"   • Colunas totais: {len(self.df.columns)}")
            print(f"   • Colunas mapeadas: {len(self.mapeamento_colunas)}")
            print(f"   • Detecção automática: {'✅ 100%' if sucesso else '⚠️  Parcial'}")
            print(f"   • Backup salvo: ✅")
            print()

            return True

        except Exception as e:
            print()
            print("╔" + "═" * 78 + "╗")
            print("║" + " ❌ ERRO AO CARREGAR ARQUIVO SAP ".center(78) + "║")
            print("╚" + "═" * 78 + "╝")
            print()
            print(f"❌ Erro: {str(e)}")
            print()

            import traceback
            print("🔍 Detalhes técnicos:")
            traceback.print_exc()

            self.logger.error(f"Erro SAP: {str(e)}")

            return False


# ══════════════════════════════════════════════════════════════════════════════
# EXECUÇÃO: CARREGAR ARQUIVO 1 (YSMM_VI_ACOMP)
# ══════════════════════════════════════════════════════════════════════════════

print()
print("⚠️  INSTRUÇÕES:")
print("   • Uma janela vai abrir para seleção do arquivo")
print("   • Selecione o arquivo YSMM_VI_ACOMP ORIGINAL (bruto) do SAP")
print("   • NÃO faça limpeza manual no arquivo")
print("   • Aceita .XLS ou .XLSX")
print()

input("👉 Pressione ENTER para iniciar...")

print()

try:
    # Inicializar carregador
    carregador_sap = CarregadorArquivo(fm)

    # Carregar YSMM_VI_ACOMP
    sucesso = carregador_sap.carregar_ysmm_vi_acomp()

    if sucesso:
        # Salvar referência global
        df_sap = carregador_sap.df

        arquivo_sap_info = {
            'df': df_sap,
            'arquivo': carregador_sap.arquivo_original,
            'mapeamento': carregador_sap.mapeamento_colunas,
            'log_deteccao': carregador_sap.log_deteccao
        }

        print()
        print("═" * 80)
        print("✅ BLOCO 2 - ARQUIVO 1 CONCLUÍDO")
        print("═" * 80)
        print()
        print("📋 PRÓXIMOS PASSOS:")
        print("-" * 80)
        print("   1. ✅ Revise a lista de colunas acima")
        print("   2. ✅ Verifique se os dados fazem sentido")
        print("   3. ✅ Confira a pasta 02_Dados_Entrada no Explorer")
        print("   4. 📤 Envie feedback:")
        print("       • 'ARQUIVO 1 OK' - para prosseguir")
        print("       • Descreva problemas - se houver erros")
        print("   5. ⏳ Aguarde BLOCO 3 (próximo arquivo)")
        print()
        print("🔗 VARIÁVEIS DISPONÍVEIS:")
        print("   • df_sap - DataFrame com dados SAP")
        print("   • arquivo_sap_info - Metadados completos")
        print("   • carregador_sap - Instância do carregador")
        print()

        fm.logger.info("BLOCO 2 - Arquivo 1 (SAP) concluído")

except KeyboardInterrupt:
    print()
    print("⚠️  Operação cancelada pelo usuário")

except Exception as e:
    print()
    print("╔" + "═" * 78 + "╗")
    print("║" + " ERRO FATAL - BLOCO 2 ".center(78) + "║")
    print("╚" + "═" * 78 + "╝")
    print()
    print(f"❌ Erro: {str(e)}")
    print()
    import traceback
    print("🔍 Detalhes técnicos:")
    traceback.print_exc()

print()
print("═" * 80)

╔══════════════════════════════════════════════════════════════════════════════╗
║                  BLOCO 2: CARREGADOR MODULAR - ARQUIVO 1/N                   ║
╚══════════════════════════════════════════════════════════════════════════════╝


⚠️  INSTRUÇÕES:
   • Uma janela vai abrir para seleção do arquivo
   • Selecione o arquivo YSMM_VI_ACOMP ORIGINAL (bruto) do SAP
   • NÃO faça limpeza manual no arquivo
   • Aceita .XLS ou .XLSX


╔══════════════════════════════════════════════════════════════════════════════╗
║               ARQUIVO 1: YSMM_VI_ACOMP (SAP - BASE HISTÓRICA)                ║
╚══════════════════════════════════════════════════════════════════════════════╝

✅ Arquivo selecionado:
   📁 Nome: 2025-2024-YSMM_VI_ACOMP.xlsx
   📊 Tamanho: 3,827,778 bytes
   🔧 Tipo: .xlsx

📖 Carregando arquivo Excel (.xlsx)...
   Tentativa 1/2: engine='openpyxl'


2025-10-15 11:01:41 | INFO     | Carregado: 2025-2024-YSMM_VI_ACOMP.xlsx (27660 × 30)
2025-10-15 11:01:41 | INFO     | Limpeza SAP aplicada: 27655 × 29
2025-10-15 11:01:41 | INFO     | Mapeamento: 3 colunas


   ✅ Sucesso! 27,660 linhas × 30 colunas

🧹 Aplicando limpeza SAP (YSMM_VI_ACOMP)...
--------------------------------------------------------------------------------
   📊 Original: 27,660 linhas × 30 colunas
   ✅ Removida coluna A (índice 0)
   ✅ Removidas linhas 1, 2, 3, 5
   ✅ Cabeçalho definido (linha 4 original)
   📊 Limpo: 27,655 linhas × 29 colunas

🔍 DETECÇÃO AUTOMÁTICA DE COLUNAS
--------------------------------------------------------------------------------
   ✅ AUTO 'Centro' → 'Centro'
           Confiança: 100.0% (match: 'Centro')
   ✅ AUTO 'Cód Grupo de produto' → 'Cód Grupo de produto'
           Confiança: 100.0% (match: 'Cód Grupo de produto')
   ✅ AUTO 'Expedição c/ Veí' → 'Expedição'
           Confiança: 100.0% (match: 'Expedição c/ Veí')
   ✅ AUTO 'Variação Interna' → 'Variação Interna'
           Confiança: 100.0% (match: 'Variação Interna')
   ✅ AUTO 'Mês do exercício' → 'Mês'
           Confiança: 100.0% (match: 'Mês do exercício')
   ✅ AUTO 'Ano do documento do 

2025-10-15 11:01:48 | INFO     | Backup: SAP_YSMM_Limpo_20251015_110105.xlsx
2025-10-15 11:01:48 | INFO     | Log detecção: deteccao_sap_20251015_110105.json
2025-10-15 11:01:48 | INFO     | BLOCO 2 - Arquivo 1 (SAP) concluído


   ✅ Backup: SAP_YSMM_Limpo_20251015_110105.xlsx
   ✅ Log detecção: deteccao_sap_20251015_110105.json

╔══════════════════════════════════════════════════════════════════════════════╗
║                     ✅ ARQUIVO SAP CARREGADO COM SUCESSO                      ║
╚══════════════════════════════════════════════════════════════════════════════╝

📊 RESUMO DO CARREGAMENTO:
--------------------------------------------------------------------------------
   • Arquivo: 2025-2024-YSMM_VI_ACOMP.xlsx
   • Registros: 27,655
   • Colunas totais: 29
   • Colunas mapeadas: 3
   • Detecção automática: ✅ 100%
   • Backup salvo: ✅


════════════════════════════════════════════════════════════════════════════════
✅ BLOCO 2 - ARQUIVO 1 CONCLUÍDO
════════════════════════════════════════════════════════════════════════════════

📋 PRÓXIMOS PASSOS:
--------------------------------------------------------------------------------
   1. ✅ Revise a lista de colunas acima
   2. ✅ Verifique se os dados fazem sent

In [19]:
"""
═══════════════════════════════════════════════════════════════════════════════
AIVI DATA INTEGRATION - BLOCO 3: CARREGADOR MODULAR - ARQUIVO 2
═══════════════════════════════════════════════════════════════════════════════
Arquivo: ysmm_centros_br.xlsx
Tipo: Tabela de Centros/Bases (Dados Complementares)

LIMPEZA:
  ✅ Primeira linha é o cabeçalho (usar header=0)
  ✅ Sem colunas iniciais a expurgar

FUNCIONALIDADES:
  ✅ Carregamento robusto (.XLS/.XLSX)
  ✅ Listagem completa de colunas
  ✅ Preview dos dados
  ✅ Backup automático
  ✅ Logging
═══════════════════════════════════════════════════════════════════════════════
"""

import pandas as pd
from pathlib import Path
import tkinter as tk
from tkinter import filedialog, messagebox

print("╔" + "═" * 78 + "╗")
print("║" + " BLOCO 3: CARREGADOR MODULAR - ARQUIVO 2/N ".center(78) + "║")
print("╚" + "═" * 78 + "╝")
print()

# ══════════════════════════════════════════════════════════════════════════════
# FUNÇÃO: Carregar ysmm_centros_br.xlsx
# ══════════════════════════════════════════════════════════════════════════════

def carregar_ysmm_centros():
    """
    Carrega arquivo ysmm_centros_br.xlsx (tabela de centros/bases).

    Características:
      - Primeira linha = cabeçalho
      - Sem limpeza de colunas necessária
      - Dados complementares para enriquecer base SAP
    """

    print("╔" + "═" * 78 + "╗")
    print("║" + " ARQUIVO 2: ysmm_centros_br.xlsx (TABELA DE CENTROS) ".center(78) + "║")
    print("╚" + "═" * 78 + "╝")
    print()

    # ══════════════════════════════════════════════════════════════════════
    # ETAPA 1: Seleção do arquivo
    # ══════════════════════════════════════════════════════════════════════

    mensagem = """
╔═══════════════════════════════════════════════════╗
║  📂 ARQUIVO: ysmm_centros_br.xlsx                 ║
╠═══════════════════════════════════════════════════╣
║                                                   ║
║  Conteúdo: Cadastro de Centros/Bases              ║
║  Uso: Enriquecimento dos dados SAP                ║
║                                                   ║
║  Campos esperados:                                ║
║  • Centro (código)                                ║
║  • Sigla                                          ║
║  • Nome/Região                                    ║
║  • Outros atributos da base                       ║
║                                                   ║
║  ⚠️  Primeira linha é o cabeçalho                 ║
║                                                   ║
╚═══════════════════════════════════════════════════╝
    """

    try:
        # Seleção GUI
        root = tk.Tk()
        root.withdraw()
        root.lift()
        root.attributes('-topmost', True)
        root.attributes('-alpha', 0.0)
        root.update()

        messagebox.showinfo(
            "[2/N] ysmm_centros_br.xlsx",
            mensagem.strip()
        )

        arquivo = filedialog.askopenfilename(
            title="[2/N] Selecione ysmm_centros_br.xlsx",
            filetypes=[
                ("Excel files", "*.xlsx *.xls"),
                ("All files", "*.*")
            ]
        )

        root.destroy()

        if not arquivo:
            raise ValueError("❌ Nenhum arquivo selecionado")

        arquivo_path = Path(arquivo)

        print(f"✅ Arquivo selecionado:")
        print(f"   📁 Nome: {arquivo_path.name}")
        print(f"   📊 Tamanho: {arquivo_path.stat().st_size:,} bytes")
        print(f"   🔧 Tipo: {arquivo_path.suffix}")
        print()

        # ══════════════════════════════════════════════════════════════════
        # ETAPA 2: Carregamento
        # ══════════════════════════════════════════════════════════════════

        extensao = arquivo_path.suffix.lower()

        print(f"📖 Carregando arquivo Excel ({extensao})...")

        # Tentar engines
        if extensao == '.xlsx':
            engines = ['openpyxl', None]
        elif extensao == '.xls':
            engines = ['xlrd', None]
        else:
            engines = [None, 'openpyxl', 'xlrd']

        df = None
        for i, engine in enumerate(engines, 1):
            try:
                if engine:
                    print(f"   Tentativa {i}/{len(engines)}: engine='{engine}'")
                    df = pd.read_excel(arquivo_path, header=0, engine=engine)
                else:
                    print(f"   Tentativa {i}/{len(engines)}: engine automático")
                    df = pd.read_excel(arquivo_path, header=0)

                print(f"   ✅ Sucesso! {df.shape[0]:,} linhas × {df.shape[1]} colunas")
                fm.logger.info(f"Carregado: {arquivo_path.name} ({df.shape[0]} × {df.shape[1]})")
                break

            except Exception as e:
                print(f"   ❌ Falhou: {str(e)[:60]}...")
                if i == len(engines):
                    if extensao == '.xls':
                        print()
                        print("💡 SOLUÇÃO para .XLS:")
                        print("   pip install xlrd")
                    raise Exception(f"Falha ao ler: {e}")

        if df is None:
            raise Exception("Nenhum engine funcionou")

        print()

        # ══════════════════════════════════════════════════════════════════
        # ETAPA 3: Limpeza (não necessária para este arquivo)
        # ══════════════════════════════════════════════════════════════════

        print("🧹 Limpeza de dados...")
        print("-" * 80)
        print("   ✅ Nenhuma limpeza necessária (primeira linha = cabeçalho)")
        print(f"   📊 Dados prontos: {df.shape[0]:,} linhas × {df.shape[1]} colunas")
        print()

        # ══════════════════════════════════════════════════════════════════
        # ETAPA 4: Listar todas as colunas
        # ══════════════════════════════════════════════════════════════════

        print("📑 LISTAGEM COMPLETA DE COLUNAS")
        print("═" * 80)
        print(f"📏 Total: {len(df.columns)} colunas")
        print()

        for i, col in enumerate(df.columns, 1):
            tipo = df[col].dtype
            nulos = df[col].isna().sum()
            pct_nulos = (nulos / len(df) * 100) if len(df) > 0 else 0
            unicos = df[col].nunique()

            print(f"   {i:2d}. {col}")
            print(f"       ├─ Tipo: {tipo}")
            print(f"       ├─ Nulos: {nulos:,} ({pct_nulos:.1f}%)")
            print(f"       └─ Únicos: {unicos:,}")

        print()

        # ══════════════════════════════════════════════════════════════════
        # ETAPA 5: Preview dos dados
        # ══════════════════════════════════════════════════════════════════

        print("📊 PREVIEW DOS DADOS (primeiras 5 colunas)")
        print("-" * 80)

        colunas_preview = list(df.columns[:5])
        print(df[colunas_preview].head(5).to_string(index=False))
        print()

        # ══════════════════════════════════════════════════════════════════
        # ETAPA 6: Backup
        # ══════════════════════════════════════════════════════════════════

        print("💾 Salvando backup...")

        timestamp = fm.timestamp
        arquivo_destino = fm.diretorios['dados_entrada'] / \
                         f"Centros_BR_{timestamp}.xlsx"

        try:
            df.to_excel(arquivo_destino, index=False, engine='openpyxl')
            fm.logger.info(f"Backup: {arquivo_destino.name}")
            print(f"   ✅ Backup salvo: {arquivo_destino.name}")
        except Exception as e:
            print(f"   ⚠️  Erro ao salvar backup: {str(e)}")
            fm.logger.error(f"Erro backup: {str(e)}")

        print()

        # ══════════════════════════════════════════════════════════════════
        # RESULTADO FINAL
        # ══════════════════════════════════════════════════════════════════

        print("╔" + "═" * 78 + "╗")
        print("║" + " ✅ ARQUIVO CENTROS CARREGADO COM SUCESSO ".center(78) + "║")
        print("╚" + "═" * 78 + "╝")
        print()

        print("📊 RESUMO DO CARREGAMENTO:")
        print("-" * 80)
        print(f"   • Arquivo: {arquivo_path.name}")
        print(f"   • Registros: {len(df):,}")
        print(f"   • Colunas: {len(df.columns)}")
        print(f"   • Backup: ✅")
        print()

        return df, arquivo_path

    except Exception as e:
        print()
        print("╔" + "═" * 78 + "╗")
        print("║" + " ❌ ERRO AO CARREGAR ARQUIVO ".center(78) + "║")
        print("╚" + "═" * 78 + "╝")
        print()
        print(f"❌ Erro: {str(e)}")
        print()

        import traceback
        print("🔍 Detalhes técnicos:")
        traceback.print_exc()

        fm.logger.error(f"Erro centros: {str(e)}")

        return None, None


# ══════════════════════════════════════════════════════════════════════════════
# EXECUÇÃO: CARREGAR ARQUIVO 2
# ══════════════════════════════════════════════════════════════════════════════

print()
print("⚠️  INSTRUÇÕES:")
print("   • Janela de seleção vai abrir")
print("   • Selecione o arquivo ysmm_centros_br.xlsx")
print("   • Primeira linha deve ser o cabeçalho")
print()

input("👉 Pressione ENTER para iniciar...")

print()

try:
    # Carregar arquivo
    df_centros, arquivo_centros = carregar_ysmm_centros()

    if df_centros is not None:
        # Salvar referência global
        arquivo_centros_info = {
            'df': df_centros,
            'arquivo': arquivo_centros
        }

        print()
        print("═" * 80)
        print("✅ BLOCO 3 - ARQUIVO 2 CONCLUÍDO")
        print("═" * 80)
        print()
        print("📋 PRÓXIMOS PASSOS:")
        print("-" * 80)
        print("   1. ✅ Revise a lista de colunas acima")
        print("   2. ✅ Verifique se os dados fazem sentido")
        print("   3. 📤 Envie feedback:")
        print("       • 'ARQUIVO 2 OK' - para prosseguir")
        print("       • Descreva problemas - se houver erros")
        print("   4. ⏳ Aguarde próximo arquivo")
        print()
        print("🔗 VARIÁVEIS DISPONÍVEIS:")
        print("   • df_centros - DataFrame com cadastro de centros")
        print("   • arquivo_centros_info - Metadados completos")
        print()

        fm.logger.info("BLOCO 3 - Arquivo 2 (Centros) concluído")

except KeyboardInterrupt:
    print()
    print("⚠️  Operação cancelada pelo usuário")

except Exception as e:
    print()
    print("╔" + "═" * 78 + "╗")
    print("║" + " ERRO FATAL - BLOCO 3 ".center(78) + "║")
    print("╚" + "═" * 78 + "╝")
    print()
    print(f"❌ Erro: {str(e)}")
    print()
    import traceback
    print("🔍 Detalhes técnicos:")
    traceback.print_exc()

print()
print("═" * 80)

╔══════════════════════════════════════════════════════════════════════════════╗
║                  BLOCO 3: CARREGADOR MODULAR - ARQUIVO 2/N                   ║
╚══════════════════════════════════════════════════════════════════════════════╝


⚠️  INSTRUÇÕES:
   • Janela de seleção vai abrir
   • Selecione o arquivo ysmm_centros_br.xlsx
   • Primeira linha deve ser o cabeçalho


╔══════════════════════════════════════════════════════════════════════════════╗
║             ARQUIVO 2: ysmm_centros_br.xlsx (TABELA DE CENTROS)              ║
╚══════════════════════════════════════════════════════════════════════════════╝



2025-10-15 11:02:04 | INFO     | Carregado: ysmm_centros_br.xlsx (555 × 30)


✅ Arquivo selecionado:
   📁 Nome: ysmm_centros_br.xlsx
   📊 Tamanho: 117,411 bytes
   🔧 Tipo: .xlsx

📖 Carregando arquivo Excel (.xlsx)...
   Tentativa 1/2: engine='openpyxl'
   ✅ Sucesso! 555 linhas × 30 colunas

🧹 Limpeza de dados...
--------------------------------------------------------------------------------
   ✅ Nenhuma limpeza necessária (primeira linha = cabeçalho)
   📊 Dados prontos: 555 linhas × 30 colunas

📑 LISTAGEM COMPLETA DE COLUNAS
════════════════════════════════════════════════════════════════════════════════
📏 Total: 30 colunas

    1. Centro
       ├─ Tipo: int64
       ├─ Nulos: 0 (0.0%)
       └─ Únicos: 555
    2. Sigla
       ├─ Tipo: object
       ├─ Nulos: 0 (0.0%)
       └─ Únicos: 499
    3. Regional/Região
       ├─ Tipo: object
       ├─ Nulos: 463 (83.4%)
       └─ Únicos: 5
    4. Nome 1
       ├─ Tipo: object
       ├─ Nulos: 0 (0.0%)
       └─ Únicos: 550
    5. Rua
       ├─ Tipo: object
       ├─ Nulos: 2 (0.4%)
       └─ Únicos: 485
    6. Número


2025-10-15 11:02:04 | INFO     | Backup: Centros_BR_20251015_110105.xlsx
2025-10-15 11:02:04 | INFO     | BLOCO 3 - Arquivo 2 (Centros) concluído


   ✅ Backup salvo: Centros_BR_20251015_110105.xlsx

╔══════════════════════════════════════════════════════════════════════════════╗
║                   ✅ ARQUIVO CENTROS CARREGADO COM SUCESSO                    ║
╚══════════════════════════════════════════════════════════════════════════════╝

📊 RESUMO DO CARREGAMENTO:
--------------------------------------------------------------------------------
   • Arquivo: ysmm_centros_br.xlsx
   • Registros: 555
   • Colunas: 30
   • Backup: ✅


════════════════════════════════════════════════════════════════════════════════
✅ BLOCO 3 - ARQUIVO 2 CONCLUÍDO
════════════════════════════════════════════════════════════════════════════════

📋 PRÓXIMOS PASSOS:
--------------------------------------------------------------------------------
   1. ✅ Revise a lista de colunas acima
   2. ✅ Verifique se os dados fazem sentido
   3. 📤 Envie feedback:
       • 'ARQUIVO 2 OK' - para prosseguir
       • Descreva problemas - se houver erros
   4. ⏳ Aguarde p

In [24]:
"""
═══════════════════════════════════════════════════════════════════
BLOCO 4: CARREGADOR AIVI OPAV BW - CORRIGIDO v3
═══════════════════════════════════════════════════════════════════
Sheet correto: "Valor da Variação Total"
Cabeçalho: L34-AM34 (APENAS, colunas AW-BH descartadas)
Resultado: 28 colunas (7 dimensões + 21 movimentações)
Tratamento dinâmico de duplicadas: sufixo _dup1, _dup2, etc
"""

import pandas as pd
import xlrd
from pathlib import Path
import tkinter as tk
from tkinter import filedialog

print("BLOCO 4: CARREGADOR AIVI OPAV BW - ARQUIVO 3/N")
print("=" * 80)
print()

# ══════════════════════════════════════════════════════════════════
# ETAPA 1: Seleção do arquivo
# ══════════════════════════════════════════════════════════════════

if 'fm' in dir():
    padrao_bw = '*xSAPtemp*.xls*'
    arquivos_encontrados = list(fm.diretorios['dados_entrada'].glob(padrao_bw))

    if arquivos_encontrados:
        arquivo_path = arquivos_encontrados[0]
        print(f"Arquivo encontrado: {arquivo_path.name}")
    else:
        root = tk.Tk()
        root.withdraw()
        root.lift()
        root.attributes('-topmost', True)

        arquivo = filedialog.askopenfilename(
            title="Selecione arquivo BW (xSAPtemp...)",
            filetypes=[("Excel", "*.xls *.xlsx"), ("All", "*.*")]
        )

        root.destroy()

        if not arquivo:
            raise ValueError("Nenhum arquivo selecionado")

        arquivo_path = Path(arquivo)
else:
    root = tk.Tk()
    root.withdraw()
    root.lift()
    root.attributes('-topmost', True)

    arquivo = filedialog.askopenfilename(
        title="Selecione arquivo BW (xSAPtemp...)",
        filetypes=[("Excel", "*.xls *.xlsx"), ("All", "*.*")]
    )

    root.destroy()

    if not arquivo:
        raise ValueError("Nenhum arquivo selecionado")

    arquivo_path = Path(arquivo)

print(f"Arquivo selecionado: {arquivo_path.name}")
print()

# ══════════════════════════════════════════════════════════════════
# ETAPA 2: Carregar SHEET CORRETO
# ══════════════════════════════════════════════════════════════════

print("ETAPA 2: Carregando sheet correto...")
print("-" * 80)

# Nome do sheet correto
SHEET_NAME = "Valor da Variação Total"

try:
    # Método 1: xlrd (mais confiável para .xls)
    print(f"Tentando xlrd (sheet: '{SHEET_NAME}')...")

    workbook = xlrd.open_workbook(str(arquivo_path))

    # Listar sheets
    print(f"   Sheets disponiveis: {workbook.sheet_names()}")

    # Verificar se sheet existe
    if SHEET_NAME not in workbook.sheet_names():
        raise ValueError(f"Sheet '{SHEET_NAME}' nao encontrado!")

    # Pegar sheet correto
    sheet = workbook.sheet_by_name(SHEET_NAME)

    print(f"   Sheet: '{SHEET_NAME}'")
    print(f"   Linhas: {sheet.nrows:,}")
    print(f"   Colunas: {sheet.ncols}")

    # Converter para lista de listas
    data = []
    for row_idx in range(sheet.nrows):
        data.append(sheet.row_values(row_idx))

    # Criar DataFrame
    df_bruto = pd.DataFrame(data)

    print(f"   DataFrame criado: {df_bruto.shape[0]:,} x {df_bruto.shape[1]}")
    print()

except Exception as e:
    print(f"   Erro com xlrd: {str(e)}")
    print()

    # Método 2: pandas como fallback
    print(f"Tentando pandas (sheet: '{SHEET_NAME}')...")

    df_bruto = pd.read_excel(
        arquivo_path,
        sheet_name=SHEET_NAME,
        header=None
    )

    print(f"   DataFrame criado: {df_bruto.shape[0]:,} x {df_bruto.shape[1]}")
    print()

# ══════════════════════════════════════════════════════════════════
# ETAPA 3: VERIFICAR LINHA 34 (índice 33)
# ══════════════════════════════════════════════════════════════════

print("ETAPA 3: Verificando linha 34 (cabecalho)...")
print("-" * 80)

# Linha 34 Excel = índice 33 pandas
linha_34 = df_bruto.iloc[33]

# Verificar colunas L-BH (índices 11-59)
cols_lbh = linha_34.iloc[11:60]

# Contar não-nulas
non_null = cols_lbh.notna().sum()
print(f"   Celulas nao-nulas em L34-BH34: {non_null}/49")

# Mostrar primeiras não-nulas
print(f"   Primeiros valores nao-nulos:")
count = 0
for i, val in enumerate(cols_lbh):
    if pd.notna(val) and count < 10:
        col_idx = 11 + i
        # Nome da coluna (L=11, M=12, etc)
        if col_idx < 26:
            col_name = chr(65 + col_idx)
        else:
            col_name = chr(65 + (col_idx // 26) - 1) + chr(65 + (col_idx % 26))

        val_str = str(val)[:30]
        print(f"      {col_name}34 (idx={col_idx}): '{val_str}'")
        count += 1

print()

# ══════════════════════════════════════════════════════════════════
# ETAPA 4: EXTRAIR CABEÇALHO CORRETO
# ══════════════════════════════════════════════════════════════════

print("ETAPA 4: Extraindo cabecalho das colunas especificas...")
print("-" * 80)

# Linha 34 Excel = índice 33
linha_cabecalho = df_bruto.iloc[33]

# APENAS PARTE 1: Colunas L-AM (índices 11-38)
# L=11, M=12, ..., AM=38 (28 colunas)
cabecalho_bruto = linha_cabecalho.iloc[11:39].tolist()

print(f"   COLUNAS EXTRAIDAS (L34-AM34): {len(cabecalho_bruto)} colunas")
print()

# Limpar nomes
cabecalho_temp = []
for i, nome in enumerate(cabecalho_bruto):
    if pd.isna(nome) or str(nome).strip() == '':
        nome_limpo = f'Col_{i}'
    else:
        # Remover apóstrofo inicial se existir
        nome_str = str(nome).strip()
        if nome_str.startswith("'"):
            nome_str = nome_str[1:]
        # Remover quebras de linha
        nome_str = nome_str.replace('\n', ' ')
        nome_limpo = nome_str
    cabecalho_temp.append(nome_limpo)

# Tratar duplicadas (adicionar sufixo _dup1, _dup2, etc)
from collections import Counter

contagem = Counter(cabecalho_temp)
duplicadas = {nome: count for nome, count in contagem.items() if count > 1}

if duplicadas:
    print(f"   AVISO: {len(duplicadas)} nomes duplicados encontrados")
    for nome, count in list(duplicadas.items())[:5]:
        print(f"      '{nome}': {count} ocorrencias")
    print()

# Renomear duplicadas
cabecalho_limpo = []
contador = {}

for nome in cabecalho_temp:
    if nome in contador:
        contador[nome] += 1
        novo_nome = f"{nome}_dup{contador[nome]}"
        cabecalho_limpo.append(novo_nome)
    else:
        contador[nome] = 0
        cabecalho_limpo.append(nome)

# Mostrar cabeçalho final
print("   Cabecalho final:")
for i, nome in enumerate(cabecalho_limpo[:10], 1):
    print(f"      A{i if i <= 9 else str(i)}: {nome}")
if len(cabecalho_limpo) > 10:
    print(f"      ... (mais {len(cabecalho_limpo) - 10} colunas)")

print()

# ══════════════════════════════════════════════════════════════════
# ETAPA 5: EXTRAIR DADOS (linhas após linha 34)
# ══════════════════════════════════════════════════════════════════

print("ETAPA 5: Extraindo dados...")
print("-" * 80)

# Dados começam na linha 35 (índice 34)
linha_inicio_dados = 34

# Extrair APENAS colunas L-AM (índices 11-38)
df_final = df_bruto.iloc[linha_inicio_dados:, 11:39].copy()

# Aplicar cabeçalho
df_final.columns = cabecalho_limpo

# Reset index
df_final = df_final.reset_index(drop=True)

print(f"   Registros: {len(df_final):,}")
print(f"   Colunas: {len(df_final.columns)}")
print()

# ══════════════════════════════════════════════════════════════════
# ETAPA 6: CONVERTER DIMENSÕES PARA STRING
# ══════════════════════════════════════════════════════════════════

print("ETAPA 6: Convertendo dimensoes para STRING...")
print("-" * 80)

# Primeiras 7 colunas = dimensões
dimensoes_cols = df_final.columns[:7].tolist()

for col in dimensoes_cols:
    if col in df_final.columns:
        df_final[col] = df_final[col].astype(str)
        print(f"   '{col}' -> STRING")

print()

# ══════════════════════════════════════════════════════════════════
# ETAPA 7: PREVIEW DOS DADOS
# ══════════════════════════════════════════════════════════════════

print("ETAPA 7: PREVIEW DOS DADOS")
print("-" * 80)
print("Primeiras 7 colunas (DIMENSOES):")
print(df_final.iloc[:, :7].head(5).to_string(index=False))
print()

print("Primeiras 5 linhas completas:")
print(df_final.head(5).to_string(index=False, max_colwidth=20))
print()

# ══════════════════════════════════════════════════════════════════
# ETAPA 8: ESTATÍSTICAS DAS COLUNAS
# ══════════════════════════════════════════════════════════════════

print("ETAPA 8: ESTATISTICAS DAS COLUNAS")
print("-" * 80)

for i, col in enumerate(df_final.columns, 1):
    # Tratar caso de coluna duplicada (retorna DataFrame)
    try:
        col_serie = df_final[col]

        # Se retornar DataFrame (duplicada), pegar primeira coluna
        if isinstance(col_serie, pd.DataFrame):
            col_serie = col_serie.iloc[:, 0]

        tipo = col_serie.dtype
        nulos = col_serie.isna().sum()
        pct_nulos = (nulos / len(df_final) * 100) if len(df_final) > 0 else 0
        unicos = col_serie.nunique()

    except Exception as e:
        # Fallback se der erro
        tipo = "unknown"
        nulos = 0
        pct_nulos = 0.0
        unicos = 0
        print(f"   AVISO: Erro ao analisar coluna '{col}': {str(e)[:50]}")

    # Identificar bloco
    if i <= 7:
        bloco = "DIMENSAO"
    elif i <= 28:
        bloco = "MOVIM_BLK1"
    else:
        bloco = "MOVIM_BLK2"

    print(f"   {i:2d}. {col[:30]}")
    print(f"       Bloco: {bloco} | Tipo: {tipo} | Nulos: {pct_nulos:.1f}% | Unicos: {unicos:,}")

print()

# ══════════════════════════════════════════════════════════════════
# ETAPA 9: SALVAR BACKUP
# ══════════════════════════════════════════════════════════════════

print("ETAPA 9: Salvando backup...")

if 'fm' in dir():
    timestamp = fm.timestamp
    arquivo_destino = fm.diretorios['dados_entrada'] / f"AIVI_OPAV_BW_{timestamp}.xlsx"

    try:
        df_final.to_excel(arquivo_destino, index=False, engine='openpyxl')
        fm.logger.info(f"Backup BW: {arquivo_destino.name}")
        print(f"   Backup: {arquivo_destino.name}")
    except Exception as e:
        print(f"   Erro ao salvar: {str(e)}")
        fm.logger.error(f"Erro backup BW: {str(e)}")
else:
    print("   FileManager nao disponivel - backup ignorado")

print()

# ══════════════════════════════════════════════════════════════════
# RESULTADO FINAL
# ══════════════════════════════════════════════════════════════════

print("=" * 80)
print("ARQUIVO BW CARREGADO E PROCESSADO COM SUCESSO")
print("=" * 80)
print()

print("RESUMO DO PROCESSAMENTO:")
print("-" * 80)
print(f"   Arquivo: {arquivo_path.name}")
print(f"   Sheet: {SHEET_NAME}")
print(f"   Registros: {len(df_final):,}")
print(f"   Colunas totais: {len(df_final.columns)}")
print(f"   Dimensoes (1-7): 7")
print(f"   Movimentacoes (8-{len(df_final.columns)}): {len(df_final.columns) - 7}")
print()

# Salvar na variável global
df_opav = df_final
arquivo_opav = arquivo_path

print("VARIAVEIS DISPONIVEIS:")
print("   df_opav - DataFrame BW processado")
print("   arquivo_opav - Path do arquivo")
print()

print("=" * 80)
print("FIM BLOCO 4")
print("=" * 80)

BLOCO 4: CARREGADOR AIVI OPAV BW - ARQUIVO 3/N

Arquivo encontrado: Cópia de xSAPtemp4687_JAN_25.xls
Arquivo selecionado: Cópia de xSAPtemp4687_JAN_25.xls

ETAPA 2: Carregando sheet correto...
--------------------------------------------------------------------------------
Tentando xlrd (sheet: 'Valor da Variação Total')...
   Sheets disponiveis: ['SAPBEXqueriesDefunct', 'SAPBEXfiltersDefunct', 'Valor da Variação Total', 'Valor da Variação Total Grupo', 'Limite Técnico', 'Justificar', 'Limite Técnico Grupo', 'BExRepositorySheet', 'Justificar Grupo', 'Custo do Produto', 'Imposto']
   Sheet: 'Valor da Variação Total'
   Linhas: 1,001
   Colunas: 60
   DataFrame criado: 1,001 x 60

ETAPA 3: Verificando linha 34 (cabecalho)...
--------------------------------------------------------------------------------
   Celulas nao-nulas em L34-BH34: 49/49
   Primeiros valores nao-nulos:
      L34 (idx=11): 'Centro de lucro'
      M34 (idx=12): 'Ano civil/mês'
      N34 (idx=13): 'Centro'
      O34 (

2025-10-15 15:02:15 | INFO     | Backup BW: AIVI_OPAV_BW_20251015_110105.xlsx


   Backup: AIVI_OPAV_BW_20251015_110105.xlsx

ARQUIVO BW CARREGADO E PROCESSADO COM SUCESSO

RESUMO DO PROCESSAMENTO:
--------------------------------------------------------------------------------
   Arquivo: Cópia de xSAPtemp4687_JAN_25.xls
   Sheet: Valor da Variação Total
   Registros: 967
   Colunas totais: 28
   Dimensoes (1-7): 7
   Movimentacoes (8-28): 21

VARIAVEIS DISPONIVEIS:
   df_opav - DataFrame BW processado
   arquivo_opav - Path do arquivo

FIM BLOCO 4


In [23]:
"""
═══════════════════════════════════════════════════════════════════
BLOCO 5: UNIFICADOR DE ARQUIVOS OPAV BW
═══════════════════════════════════════════════════════════════════
Busca recursiva: *xSAPtemp*.xls* em pasta e subpastas
Carrega cada arquivo com mesmo padrão do BLOCO 4
Unifica tudo em um único DataFrame
"""

import pandas as pd
import xlrd
from pathlib import Path
from collections import Counter
import tkinter as tk
from tkinter import filedialog

print("╔" + "═" * 78 + "╗")
print("║" + " BLOCO 5: UNIFICADOR DE ARQUIVOS OPAV BW ".center(78) + "║")
print("╚" + "═" * 78 + "╝")
print()

# ══════════════════════════════════════════════════════════════════
# CONFIGURAÇÕES
# ══════════════════════════════════════════════════════════════════

SHEET_NAME = "Valor da Variação Total"
LINHA_CABECALHO = 33  # Linha 34 do Excel
LINHA_INICIO_DADOS = 34

# Colunas a extrair (APENAS L-AM, 28 colunas)
COLS_EXTRAIR = (11, 39)  # L-AM (índices 11-38)

# ══════════════════════════════════════════════════════════════════
# FUNÇÕES AUXILIARES (do BLOCO 4)
# ══════════════════════════════════════════════════════════════════

def limpar_nome_coluna(nome):
    """
    Limpa nome de coluna de caracteres especiais
    """
    if pd.isna(nome) or str(nome).strip() == '':
        return None

    nome_str = str(nome).strip()
    nome_str = nome_str.lstrip("'")
    nome_str = nome_str.replace('\n', ' ')
    nome_str = nome_str.replace('\r', ' ')
    nome_str = ' '.join(nome_str.split())

    return nome_str

def tratar_duplicadas(colunas):
    """
    Adiciona sufixo _dup1, _dup2 às duplicadas
    """
    contagem = Counter(colunas)
    duplicadas = {n: c for n, c in contagem.items() if c > 1}

    if duplicadas:
        print(f"   AVISO: {len(duplicadas)} nomes duplicados")
        for nome, count in list(duplicadas.items())[:3]:
            print(f"      '{nome}': {count} ocorrencias")

    colunas_novas = []
    contador = {}

    for nome in colunas:
        if nome in contador:
            contador[nome] += 1
            colunas_novas.append(f"{nome}_dup{contador[nome]}")
        else:
            contador[nome] = 0
            colunas_novas.append(nome)

    return colunas_novas

def carregar_arquivo_opav(arquivo_path):
    """
    Carrega um arquivo OPAV seguindo padrão do BLOCO 4

    Retorna: DataFrame ou None se erro
    """
    try:
        print(f"\n📄 Processando: {arquivo_path.name}")
        print("   " + "-" * 76)

        # Carregar com xlrd
        workbook = xlrd.open_workbook(str(arquivo_path))

        # Verificar sheet
        if SHEET_NAME not in workbook.sheet_names():
            print(f"   ❌ Sheet '{SHEET_NAME}' nao encontrado")
            print(f"   Sheets disponiveis: {workbook.sheet_names()}")
            return None

        sheet = workbook.sheet_by_name(SHEET_NAME)
        print(f"   ✅ Sheet: '{SHEET_NAME}' ({sheet.nrows} linhas × {sheet.ncols} cols)")

        # Converter para DataFrame
        data = [sheet.row_values(i) for i in range(sheet.nrows)]
        df_bruto = pd.DataFrame(data)

        # Extrair cabeçalho (APENAS L-AM)
        linha_cab = df_bruto.iloc[LINHA_CABECALHO]
        cab_bruto = linha_cab.iloc[COLS_EXTRAIR[0]:COLS_EXTRAIR[1]].tolist()

        # Limpar cabeçalho
        cab_temp = [limpar_nome_coluna(c) or f'Col_{i}'
                    for i, c in enumerate(cab_bruto)]

        # Tratar duplicadas
        cab_final = tratar_duplicadas(cab_temp)

        # Extrair dados (APENAS L-AM)
        df_final = df_bruto.iloc[LINHA_INICIO_DADOS:, COLS_EXTRAIR[0]:COLS_EXTRAIR[1]].copy()

        df_final.columns = cab_final
        df_final = df_final.reset_index(drop=True)

        # Adicionar metadados
        df_final['_arquivo_origem'] = arquivo_path.name
        df_final['_arquivo_path'] = str(arquivo_path)
        df_final['_data_carga'] = pd.Timestamp.now()

        print(f"   ✅ Carregado: {len(df_final):,} registros × {len(cab_final)} colunas")

        return df_final

    except Exception as e:
        print(f"   ❌ ERRO: {str(e)[:100]}")
        import traceback
        print(f"   Detalhes: {traceback.format_exc()[:200]}")
        return None

# ══════════════════════════════════════════════════════════════════
# ETAPA 1: Determinar pasta base
# ══════════════════════════════════════════════════════════════════

print("ETAPA 1: Determinar pasta com arquivos OPAV")
print("-" * 80)
print()

pasta_base = None

# OPÇÃO 1: Usar pasta do arquivo já carregado (BLOCO 4)
if 'arquivo_opav' in dir():
    pasta_arquivo = arquivo_opav.parent
    print(f"📁 Arquivo OPAV ja carregado no BLOCO 4:")
    print(f"   Arquivo: {arquivo_opav.name}")
    print(f"   Pasta: {pasta_arquivo}")
    print()

    # Verificar se há outros arquivos nessa pasta
    outros_arquivos = list(pasta_arquivo.glob('*xSAPtemp*.xls*'))

    if len(outros_arquivos) > 1:
        print(f"   ✅ Encontrados {len(outros_arquivos)} arquivos xSAPtemp nesta pasta")
        print(f"   Usando esta pasta como base")
        pasta_base = pasta_arquivo
    elif len(outros_arquivos) == 1:
        print(f"   ⚠️  Apenas 1 arquivo xSAPtemp nesta pasta")
        print(f"   Vou perguntar se quer buscar em outra pasta")
    else:
        print(f"   ⚠️  Nenhum arquivo xSAPtemp nesta pasta")
        print(f"   Vou perguntar outra pasta")

# OPÇÃO 2: Perguntar ao usuário
if pasta_base is None:
    print()
    print("═" * 80)
    print("📂 SELEÇÃO DE PASTA")
    print("═" * 80)
    print()
    print("Opcoes:")
    print("   1. Selecionar pasta manualmente")
    if 'arquivo_opav' in dir():
        print(f"   2. Usar pasta do arquivo atual ({arquivo_opav.parent.name})")
    if 'fm' in dir():
        print(f"   3. Usar pasta do FileManager ({fm.diretorios['dados_entrada'].name})")
    print()

    escolha = input("Escolha (1/2/3): ").strip()

    if escolha == '1':
        # Janela de seleção
        print("\nAbrindo janela de selecao de pasta...")

        root = tk.Tk()
        root.withdraw()
        root.lift()
        root.attributes('-topmost', True)

        pasta = filedialog.askdirectory(
            title="Selecione pasta com arquivos OPAV (xSAPtemp...)"
        )

        root.destroy()

        if not pasta:
            raise ValueError("Nenhuma pasta selecionada")

        pasta_base = Path(pasta)

    elif escolha == '2' and 'arquivo_opav' in dir():
        pasta_base = arquivo_opav.parent

    elif escolha == '3' and 'fm' in dir():
        pasta_base = fm.diretorios['dados_entrada']

    else:
        # Default: FileManager ou erro
        if 'fm' in dir():
            pasta_base = fm.diretorios['dados_entrada']
        else:
            raise ValueError("Opcao invalida")

# ══════════════════════════════════════════════════════════════════
# ETAPA 1.5: Copiar arquivo para pasta FileManager (se necessário)
# ══════════════════════════════════════════════════════════════════

if 'arquivo_opav' in dir() and 'fm' in dir():
    # Verificar se arquivo está fora da pasta do FileManager
    arquivo_atual = arquivo_opav
    pasta_fm = fm.diretorios['dados_entrada']

    if arquivo_atual.parent != pasta_fm:
        print()
        print("═" * 80)
        print("📋 ARQUIVO FORA DA PASTA DO FILEMANAGER")
        print("═" * 80)
        print()
        print(f"Arquivo atual: {arquivo_atual}")
        print(f"Pasta FileManager: {pasta_fm}")
        print()

        # Verificar tamanho
        tamanho_mb = arquivo_atual.stat().st_size / (1024 * 1024)
        print(f"Tamanho do arquivo: {tamanho_mb:.2f} MB")
        print()

        if tamanho_mb > 50:
            print("⚠️  Arquivo grande (>50MB)")
            print()
            print("Opcoes:")
            print("   1. Copiar para pasta FileManager (pode demorar)")
            print("   2. Buscar na pasta atual do arquivo")
            print("   3. Selecionar outra pasta")
            print()

            escolha_copia = input("Escolha (1/2/3): ").strip()

            if escolha_copia == '1':
                print("\n📋 Copiando arquivo...")
                import shutil

                destino = pasta_fm / arquivo_atual.name

                try:
                    shutil.copy2(arquivo_atual, destino)
                    print(f"   ✅ Copiado para: {destino}")
                    print()
                except Exception as e:
                    print(f"   ❌ Erro ao copiar: {str(e)}")
                    print(f"   Usando pasta original")
                    print()

            elif escolha_copia == '2':
                print("\n✅ Usando pasta do arquivo atual")
                print()

            elif escolha_copia == '3':
                print("\n📂 Abrindo janela de selecao...")
                root = tk.Tk()
                root.withdraw()
                root.lift()
                root.attributes('-topmost', True)

                pasta = filedialog.askdirectory(
                    title="Selecione pasta com arquivos OPAV"
                )

                root.destroy()

                if pasta:
                    pasta_base = Path(pasta)
                    print(f"   ✅ Pasta selecionada: {pasta_base}")
                print()
        else:
            # Arquivo pequeno (<50MB), copiar automaticamente
            print("✅ Arquivo pequeno (<50MB)")
            print("   Copiando para pasta FileManager...")

            import shutil
            destino = pasta_fm / arquivo_atual.name

            try:
                shutil.copy2(arquivo_atual, destino)
                print(f"   ✅ Copiado para: {destino.name}")
                print()
            except Exception as e:
                print(f"   ⚠️  Erro ao copiar: {str(e)}")
                print(f"   Continuando com pasta original")
                print()

print()

print("=" * 80)
print(f"✅ PASTA BASE DEFINIDA: {pasta_base}")
print("=" * 80)
print()

# ══════════════════════════════════════════════════════════════════
# ETAPA 2: Buscar arquivos recursivamente
# ══════════════════════════════════════════════════════════════════

print("ETAPA 2: Buscando arquivos xSAPtemp recursivamente...")
print("-" * 80)
print()

# Buscar com glob recursivo
padroes = ['*xSAPtemp*.xls', '*xSAPtemp*.xlsx']
arquivos_encontrados = []

for padrao in padroes:
    arquivos_encontrados.extend(pasta_base.rglob(padrao))

# Remover duplicatas (caso encontre mesmo arquivo com ambos padrões)
arquivos_encontrados = list(set(arquivos_encontrados))

# Ordenar por nome
arquivos_encontrados.sort(key=lambda x: x.name)

print(f"✅ Encontrados: {len(arquivos_encontrados)} arquivos")
print()

if len(arquivos_encontrados) == 0:
    print("❌ NENHUM ARQUIVO ENCONTRADO!")
    print()
    print("Padrões buscados:")
    for padrao in padroes:
        print(f"   • {padrao}")
    print()
    print(f"Pasta base: {pasta_base}")
    print()
    raise ValueError("Nenhum arquivo xSAPtemp encontrado")

# Listar arquivos encontrados
print("📋 ARQUIVOS ENCONTRADOS:")
print("-" * 80)
for i, arq in enumerate(arquivos_encontrados, 1):
    # Caminho relativo à pasta base
    rel_path = arq.relative_to(pasta_base)
    tamanho_mb = arq.stat().st_size / (1024 * 1024)
    print(f"   {i:2d}. {arq.name}")
    print(f"       Pasta: {rel_path.parent}")
    print(f"       Tamanho: {tamanho_mb:.2f} MB")

print()

# Confirmação do usuário
print("═" * 80)
print("⚠️  CONFIRMAÇÃO NECESSÁRIA")
print("═" * 80)
print()
print(f"Foram encontrados {len(arquivos_encontrados)} arquivos.")
print("Todos serão processados e unificados em um único DataFrame.")
print()
print("Deseja prosseguir?")
print("   [ENTER] = SIM, continuar")
print("   [Ctrl+C] = NÃO, cancelar")
print()

input("Pressione ENTER para continuar...")
print()

# ══════════════════════════════════════════════════════════════════
# ETAPA 3: Carregar todos os arquivos
# ══════════════════════════════════════════════════════════════════

print("ETAPA 3: Carregando todos os arquivos...")
print("═" * 80)

dataframes = []
erros = []

for i, arquivo in enumerate(arquivos_encontrados, 1):
    print(f"\n[{i}/{len(arquivos_encontrados)}] Processando...")

    df = carregar_arquivo_opav(arquivo)

    if df is not None:
        dataframes.append(df)
    else:
        erros.append(arquivo.name)

print()
print("=" * 80)
print(f"RESUMO DO CARREGAMENTO:")
print("=" * 80)
print(f"   ✅ Sucesso: {len(dataframes)} arquivos")
print(f"   ❌ Erros: {len(erros)} arquivos")

if erros:
    print(f"\n   Arquivos com erro:")
    for erro in erros:
        print(f"      • {erro}")

print()

if len(dataframes) == 0:
    raise ValueError("Nenhum arquivo foi carregado com sucesso!")

# ══════════════════════════════════════════════════════════════════
# ETAPA 4: Unificar DataFrames
# ══════════════════════════════════════════════════════════════════

print("ETAPA 4: Unificando DataFrames...")
print("-" * 80)
print()

# Concatenar verticalmente
df_unificado = pd.concat(dataframes, ignore_index=True)

print(f"✅ DataFrame unificado criado")
print(f"   Registros totais: {len(df_unificado):,}")
print(f"   Colunas: {len(df_unificado.columns)}")
print()

# Estatísticas por arquivo
print("📊 REGISTROS POR ARQUIVO:")
print("-" * 80)

contagem_por_arquivo = df_unificado['_arquivo_origem'].value_counts().sort_index()

for arquivo, count in contagem_por_arquivo.items():
    pct = (count / len(df_unificado)) * 100
    print(f"   {arquivo}: {count:,} registros ({pct:.1f}%)")

print()

# ══════════════════════════════════════════════════════════════════
# ETAPA 5: Validações
# ══════════════════════════════════════════════════════════════════

print("ETAPA 5: Validações...")
print("-" * 80)
print()

# Verificar duplicatas
print("🔍 Verificando duplicatas...")

# Colunas chave (sem metadados)
colunas_chave = [c for c in df_unificado.columns
                 if not c.startswith('_')][:7]

print(f"   Colunas chave: {colunas_chave[:5]}...")

duplicatas = df_unificado.duplicated(subset=colunas_chave)
n_duplicatas = duplicatas.sum()

if n_duplicatas > 0:
    print(f"   ⚠️  {n_duplicatas:,} linhas duplicadas encontradas")
    print(f"   Mantendo primeira ocorrência...")
    df_unificado = df_unificado[~duplicatas].reset_index(drop=True)
    print(f"   ✅ Após remover: {len(df_unificado):,} registros")
else:
    print(f"   ✅ Nenhuma duplicata encontrada")

print()

# Valores nulos em colunas críticas
print("🔍 Verificando nulos em colunas críticas...")

colunas_criticas = ['Centro', 'Produto', 'Ano civil/mês']

for col in colunas_criticas:
    if col in df_unificado.columns:
        nulos = df_unificado[col].isna().sum()
        if nulos > 0:
            print(f"   ⚠️  '{col}': {nulos:,} nulos")
        else:
            print(f"   ✅ '{col}': 0 nulos")

print()

# ══════════════════════════════════════════════════════════════════
# ETAPA 6: Preview e Estatísticas
# ══════════════════════════════════════════════════════════════════

print("ETAPA 6: PREVIEW E ESTATISTICAS")
print("═" * 80)
print()

print("Primeiras 7 colunas (DIMENSOES):")
colunas_dim = [c for c in df_unificado.columns if not c.startswith('_')][:7]
print(df_unificado[colunas_dim].head(5).to_string(index=False))
print()

print("Ultimas 3 colunas (METADADOS):")
print(df_unificado[['_arquivo_origem', '_arquivo_path', '_data_carga']].head(3).to_string(index=False))
print()

# Estatísticas resumidas
print("📊 ESTATISTICAS GERAIS:")
print("-" * 80)

print(f"   Registros totais: {len(df_unificado):,}")
print(f"   Colunas totais: {len(df_unificado.columns)}")
print(f"   Arquivos unificados: {len(dataframes)}")
print(f"   Período de carga: {df_unificado['_data_carga'].min()} a {df_unificado['_data_carga'].max()}")

# Dimensões únicas
if 'Centro' in df_unificado.columns:
    print(f"   Centros únicos: {df_unificado['Centro'].nunique()}")

if 'Produto' in df_unificado.columns:
    print(f"   Produtos únicos: {df_unificado['Produto'].nunique()}")

if 'Ano civil/mês' in df_unificado.columns:
    print(f"   Períodos únicos: {df_unificado['Ano civil/mês'].nunique()}")

print()

# ══════════════════════════════════════════════════════════════════
# ETAPA 7: Salvar Backup
# ══════════════════════════════════════════════════════════════════

print("ETAPA 7: Salvando backup...")
print("-" * 80)
print()

if 'fm' in dir():
    timestamp = fm.timestamp
    arquivo_destino = fm.diretorios['dados_processados'] / f"AIVI_OPAV_UNIFICADO_{timestamp}.xlsx"

    try:
        df_unificado.to_excel(arquivo_destino, index=False, engine='openpyxl')
        fm.logger.info(f"OPAV unificado: {arquivo_destino.name} ({len(df_unificado)} registros)")
        print(f"   ✅ Backup: {arquivo_destino.name}")
        print(f"   📁 Local: {arquivo_destino.parent}")
    except Exception as e:
        print(f"   ⚠️  Erro ao salvar: {str(e)}")
        fm.logger.error(f"Erro backup OPAV unificado: {str(e)}")
else:
    print("   ⚠️  FileManager não disponível - backup ignorado")

print()

# ══════════════════════════════════════════════════════════════════
# RESULTADO FINAL
# ══════════════════════════════════════════════════════════════════

print("╔" + "═" * 78 + "╗")
print("║" + " ✅ UNIFICAÇÃO CONCLUÍDA COM SUCESSO ".center(78) + "║")
print("╚" + "═" * 78 + "╝")
print()

print("RESUMO FINAL:")
print("-" * 80)
print(f"   📄 Arquivos processados: {len(dataframes)}/{len(arquivos_encontrados)}")
print(f"   📊 Registros totais: {len(df_unificado):,}")
print(f"   📋 Colunas: {len(df_unificado.columns)}")
print(f"   🗂️  Estrutura:")
print(f"       • Dimensões: 7")
print(f"       • Movimentações: {len(df_unificado.columns) - 10}")
print(f"       • Metadados: 3")
print()

# Salvar na variável global
df_opav_unificado = df_unificado
arquivos_opav_processados = [arq.name for arq in arquivos_encontrados if carregar_arquivo_opav(arq) is not None]

print("VARIAVEIS DISPONIVEIS:")
print("   df_opav_unificado - DataFrame unificado")
print("   arquivos_opav_processados - Lista de arquivos")
print()

print("=" * 80)
print("FIM BLOCO 5")
print("=" * 80)

╔══════════════════════════════════════════════════════════════════════════════╗
║                   BLOCO 5: UNIFICADOR DE ARQUIVOS OPAV BW                    ║
╚══════════════════════════════════════════════════════════════════════════════╝

ETAPA 1: Determinar pasta com arquivos OPAV
--------------------------------------------------------------------------------

📁 Arquivo OPAV ja carregado no BLOCO 4:
   Arquivo: Cópia de xSAPtemp4687_JAN_25.xls
   Pasta: E:\OneDrive - VIBRA\NMCV - Documentos\Indicador\AIVI\AIVI-INTEGRAÇÃO\AIVI_DataIntegration_20251015_110105\02_Dados_Entrada

   ⚠️  Apenas 1 arquivo xSAPtemp nesta pasta
   Vou perguntar se quer buscar em outra pasta

════════════════════════════════════════════════════════════════════════════════
📂 SELEÇÃO DE PASTA
════════════════════════════════════════════════════════════════════════════════

Opcoes:
   1. Selecionar pasta manualmente
   2. Usar pasta do arquivo atual (02_Dados_Entrada)
   3. Usar pasta do FileManager (02_Dado

2025-10-15 11:33:03 | INFO     | OPAV unificado: AIVI_OPAV_UNIFICADO_20251015_110105.xlsx (200 registros)


   ✅ Sheet: 'Valor da Variação Total' (1001 linhas × 60 cols)
   ✅ Carregado: 967 registros × 28 colunas

RESUMO DO CARREGAMENTO:
   ✅ Sucesso: 1 arquivos
   ❌ Erros: 0 arquivos

ETAPA 4: Unificando DataFrames...
--------------------------------------------------------------------------------

✅ DataFrame unificado criado
   Registros totais: 967
   Colunas: 31

📊 REGISTROS POR ARQUIVO:
--------------------------------------------------------------------------------
   Cópia de xSAPtemp4687_JAN_25.xls: 967 registros (100.0%)

ETAPA 5: Validações...
--------------------------------------------------------------------------------

🔍 Verificando duplicatas...
   Colunas chave: ['Centro de lucro', 'Ano civil/mês', 'Centro', 'Col_3', 'HierarqPrd']...
   ⚠️  767 linhas duplicadas encontradas
   Mantendo primeira ocorrência...
   ✅ Após remover: 200 registros

🔍 Verificando nulos em colunas críticas...
   ✅ 'Centro': 0 nulos
   ✅ 'Produto': 0 nulos
   ✅ 'Ano civil/mês': 0 nulos

ETAPA 6: PREV

In [25]:
# ═══════════════════════════════════════════════════════════════════
# BLOCO 0A: DETECTOR DE ARQUIVO DESCONHECIDO
# Sistema inteligente para processar arquivos Excel de estrutura desconhecida
# ═══════════════════════════════════════════════════════════════════

import pandas as pd
import xlrd
import re
from pathlib import Path
from collections import Counter
import numpy as np
from datetime import datetime

class DetectorArquivoDesconhecido:
    """
    Sistema para detectar automaticamente estrutura de arquivos Excel
    """

    def __init__(self, arquivo_path, fm=None):
        self.arquivo_path = Path(arquivo_path)
        self.fm = fm
        self.nome_arquivo = self.arquivo_path.name
        self.workbook = None
        self.sheet_detectada = None
        self.linha_cabecalho = None
        self.linha_dados_inicio = None
        self.df_bruto = None
        self.df_limpo = None
        self.log = []

    def processar(self):
        """
        Processamento completo do arquivo
        """
        self._log("🔍 INICIANDO DETECÇÃO AUTOMÁTICA", nivel="TITULO")
        self._log(f"📁 Arquivo: {self.nome_arquivo}")

        # 1. Carregar workbook
        self._carregar_workbook()

        # 2. Detectar sheet correto
        self._detectar_sheet()

        # 3. Detectar linha de cabeçalho
        self._detectar_cabecalho()

        # 4. Extrair e limpar dados
        self._extrair_dados()

        # 5. Limpar estrutura
        self._limpar_estrutura()

        # 6. Relatório final
        self._gerar_relatorio()

        return self.df_limpo

    def _carregar_workbook(self):
        """
        Carrega workbook Excel
        """
        self._log("\n📂 FASE 1: Carregamento do Arquivo", nivel="SECAO")

        try:
            # Tentar xlrd primeiro (mais robusto para .xls)
            self.workbook = xlrd.open_workbook(str(self.arquivo_path))
            self._log(f"✅ Carregado com xlrd")
            self._log(f"   Formato: XLS (Excel Antigo)")
        except Exception as e1:
            # Fallback para pandas (xlsx, xlsm)
            try:
                self.workbook = pd.ExcelFile(str(self.arquivo_path))
                self._log(f"✅ Carregado com pandas")
                self._log(f"   Formato: XLSX/XLSM (Excel Moderno)")
            except Exception as e2:
                raise ValueError(f"❌ Não foi possível carregar arquivo:\n  xlrd: {e1}\n  pandas: {e2}")

        # Listar sheets
        if isinstance(self.workbook, xlrd.Book):
            sheets = self.workbook.sheet_names()
        else:
            sheets = self.workbook.sheet_names

        self._log(f"📊 Sheets encontradas: {len(sheets)}")
        for i, sheet in enumerate(sheets, 1):
            self._log(f"   {i}. {sheet}")

    def _detectar_sheet(self):
        """
        Detecta sheet com dados relevantes
        """
        self._log("\n📊 FASE 2: Detecção de Sheet", nivel="SECAO")

        if isinstance(self.workbook, xlrd.Book):
            sheets = self.workbook.sheet_names()
        else:
            sheets = self.workbook.sheet_names

        # REGRAS DE DETECÇÃO (ordem de prioridade)
        regras = [
            # BW específico
            (r"Valor da Variação Total", "BW - Variação Total"),
            (r"Variação.*Total", "Variação Total (genérico)"),
            (r"OPAV", "OPAV"),
            # Genéricas
            (r"(?i)dados", "Dados"),
            (r"(?i)relat[oó]rio", "Relatório"),
            (r"(?i)export", "Export"),
            (r"(?i)result", "Result"),
        ]

        candidatos = []

        for sheet_name in sheets:
            for padrao, descricao in regras:
                if re.search(padrao, sheet_name):
                    # Carregar amostra para validar
                    validacao = self._validar_sheet(sheet_name)
                    candidatos.append({
                        'nome': sheet_name,
                        'descricao': descricao,
                        'score': validacao['score'],
                        'linhas': validacao['linhas'],
                        'colunas': validacao['colunas']
                    })
                    self._log(f"   ✅ Candidato: '{sheet_name}' ({descricao})")
                    self._log(f"      Score: {validacao['score']:.2f} | Linhas: {validacao['linhas']} | Colunas: {validacao['colunas']}")
                    break

        if not candidatos:
            # Se nenhum match, usar primeira sheet não vazia
            self._log("   ⚠️  Nenhum match por padrão, analisando todas sheets...")
            for sheet_name in sheets:
                validacao = self._validar_sheet(sheet_name)
                if validacao['score'] > 0:
                    candidatos.append({
                        'nome': sheet_name,
                        'descricao': 'Primeira não vazia',
                        'score': validacao['score'],
                        'linhas': validacao['linhas'],
                        'colunas': validacao['colunas']
                    })

        if not candidatos:
            raise ValueError("❌ Nenhuma sheet com dados foi encontrada")

        # Selecionar melhor candidato (maior score)
        melhor = max(candidatos, key=lambda x: x['score'])
        self.sheet_detectada = melhor['nome']

        self._log(f"\n   🎯 SHEET SELECIONADA: '{self.sheet_detectada}'")
        self._log(f"      {melhor['descricao']} | Score: {melhor['score']:.2f}")

    def _validar_sheet(self, sheet_name):
        """
        Valida se sheet contém dados úteis
        """
        try:
            if isinstance(self.workbook, xlrd.Book):
                sheet = self.workbook.sheet_by_name(sheet_name)
                linhas = sheet.nrows
                colunas = sheet.ncols
                # Amostra: primeira linha não vazia
                amostra = []
                for i in range(min(50, linhas)):
                    row = sheet.row_values(i)
                    if any(str(c).strip() for c in row):
                        amostra.append(row)
                        if len(amostra) >= 10:
                            break
            else:
                df_sample = pd.read_excel(self.workbook, sheet_name=sheet_name, nrows=50)
                linhas = len(df_sample)
                colunas = len(df_sample.columns)
                amostra = df_sample.values.tolist()

            # Calcular score
            score = 0.0
            if linhas > 10:
                score += 1.0
            if colunas > 5:
                score += 1.0
            if linhas > 100:
                score += 0.5
            if colunas > 15:
                score += 0.5

            # Penalizar sheets muito pequenas
            if linhas < 5 or colunas < 3:
                score = 0.0

            return {
                'score': score,
                'linhas': linhas,
                'colunas': colunas
            }
        except:
            return {'score': 0.0, 'linhas': 0, 'colunas': 0}

    def _detectar_cabecalho(self):
        """
        Detecta linha de cabeçalho automaticamente
        """
        self._log("\n🔍 FASE 3: Detecção de Cabeçalho", nivel="SECAO")

        # Carregar primeiras 100 linhas
        if isinstance(self.workbook, xlrd.Book):
            sheet = self.workbook.sheet_by_name(self.sheet_detectada)
            linhas_amostra = []
            for i in range(min(100, sheet.nrows)):
                linhas_amostra.append(sheet.row_values(i))
        else:
            df_amostra = pd.read_excel(
                self.workbook,
                sheet_name=self.sheet_detectada,
                nrows=100,
                header=None
            )
            linhas_amostra = df_amostra.values.tolist()

        # Análise linha por linha
        scores = []
        for idx, linha in enumerate(linhas_amostra):
            score = self._avaliar_linha_cabecalho(linha, idx)
            scores.append({
                'linha': idx,
                'score': score,
                'conteudo_sample': linha[:5]  # Primeiras 5 colunas
            })

        # Ordenar por score
        scores_ordenados = sorted(scores, key=lambda x: x['score'], reverse=True)

        # Mostrar top 5
        self._log("   🏆 Top 5 candidatos a cabeçalho:")
        for i, item in enumerate(scores_ordenados[:5], 1):
            self._log(f"      {i}. Linha {item['linha']+1} (Excel) - Score: {item['score']:.2f}")
            self._log(f"         Sample: {item['conteudo_sample']}")

        # Selecionar melhor
        melhor = scores_ordenados[0]
        self.linha_cabecalho = melhor['linha']
        self.linha_dados_inicio = self.linha_cabecalho + 1

        self._log(f"\n   🎯 CABEÇALHO DETECTADO: Linha {self.linha_cabecalho + 1} (Excel)")
        self._log(f"      Início dos dados: Linha {self.linha_dados_inicio + 1} (Excel)")

    def _avaliar_linha_cabecalho(self, linha, idx):
        """
        Avalia se uma linha é candidata a cabeçalho
        """
        score = 0.0

        # Converter para strings
        celulas = [str(c).strip() for c in linha if str(c).strip()]

        if not celulas:
            return 0.0

        # CRITÉRIO 1: Quantidade de células não vazias (peso 2.0)
        prop_nao_vazias = len(celulas) / len(linha)
        score += prop_nao_vazias * 2.0

        # CRITÉRIO 2: Células com texto (não só números) (peso 1.5)
        tem_texto = sum(1 for c in celulas if re.search(r'[a-zA-Z]', c))
        prop_texto = tem_texto / len(celulas) if celulas else 0
        score += prop_texto * 1.5

        # CRITÉRIO 3: Palavras-chave de cabeçalho (peso 3.0)
        keywords = [
            'centro', 'produto', 'material', 'código', 'cod', 'cód',
            'data', 'período', 'ano', 'mês', 'mes',
            'quantidade', 'valor', 'volume', 'expedição', 'variação',
            'limite', 'batente', 'sigla', 'base', 'região', 'regiao',
            'nome', 'descrição', 'descricao', 'tipo', 'status'
        ]

        texto_linha = ' '.join(celulas).lower()
        keywords_encontradas = sum(1 for kw in keywords if kw in texto_linha)
        score += (keywords_encontradas / len(keywords)) * 3.0

        # CRITÉRIO 4: Tamanho médio das células (cabeçalhos tendem a ser curtos) (peso 1.0)
        tamanho_medio = np.mean([len(c) for c in celulas])
        if 5 <= tamanho_medio <= 50:
            score += 1.0
        elif tamanho_medio > 100:
            score -= 0.5  # Penalizar linhas muito longas

        # CRITÉRIO 5: Unicidade (cabeçalhos não devem ter repetições) (peso 1.5)
        contador = Counter(celulas)
        repeticoes = sum(1 for c, n in contador.items() if n > 1)
        if repeticoes == 0:
            score += 1.5
        else:
            score -= repeticoes * 0.3

        # CRITÉRIO 6: Posição (linhas mais acima têm vantagem) (peso 0.5)
        if idx < 50:
            score += (50 - idx) / 100

        # CRITÉRIO 7: Formato típico BW (linha ~30-40)
        if 30 <= idx <= 40:
            score += 0.5

        return score

    def _extrair_dados(self):
        """
        Extrai dados a partir da linha detectada
        """
        self._log("\n📊 FASE 4: Extração de Dados", nivel="SECAO")

        if isinstance(self.workbook, xlrd.Book):
            sheet = self.workbook.sheet_by_name(self.sheet_detectada)

            # Extrair todas as linhas
            data = []
            for i in range(sheet.nrows):
                data.append(sheet.row_values(i))

            self.df_bruto = pd.DataFrame(data)

            # Definir cabeçalho
            cabecalho_bruto = self.df_bruto.iloc[self.linha_cabecalho].tolist()

            # Extrair dados
            self.df_bruto = self.df_bruto.iloc[self.linha_dados_inicio:].copy()
            self.df_bruto.columns = cabecalho_bruto

        else:
            # pandas
            self.df_bruto = pd.read_excel(
                self.workbook,
                sheet_name=self.sheet_detectada,
                header=self.linha_cabecalho
            )

        self.df_bruto = self.df_bruto.reset_index(drop=True)

        self._log(f"✅ Dados extraídos")
        self._log(f"   Registros: {len(self.df_bruto):,}")
        self._log(f"   Colunas: {len(self.df_bruto.columns)}")

    def _limpar_estrutura(self):
        """
        Limpa estrutura do DataFrame
        """
        self._log("\n🧹 FASE 5: Limpeza de Estrutura", nivel="SECAO")

        df = self.df_bruto.copy()

        # 1. Remover colunas completamente vazias
        colunas_vazias = df.columns[df.isna().all()].tolist()
        if colunas_vazias:
            self._log(f"   🗑️  Removendo {len(colunas_vazias)} colunas vazias")
            df = df.drop(columns=colunas_vazias)

        # 2. Remover linhas completamente vazias
        linhas_vazias = df.index[df.isna().all(axis=1)].tolist()
        if linhas_vazias:
            self._log(f"   🗑️  Removendo {len(linhas_vazias)} linhas vazias")
            df = df.dropna(how='all')

        # 3. Limpar nomes de colunas
        self._log("   🧹 Limpando nomes de colunas...")
        colunas_limpas = []
        for col in df.columns:
            col_limpo = str(col).strip()
            col_limpo = col_limpo.lstrip("'")  # Excel adiciona '
            col_limpo = col_limpo.replace('\n', ' ')
            col_limpo = col_limpo.replace('\r', '')
            col_limpo = ' '.join(col_limpo.split())  # Múltiplos espaços
            colunas_limpas.append(col_limpo)

        df.columns = colunas_limpas

        # 4. Renomear colunas duplicadas
        contagem = Counter(colunas_limpas)
        duplicadas = {c: n for c, n in contagem.items() if n > 1}

        if duplicadas:
            self._log(f"   ⚠️  Renomeando {len(duplicadas)} colunas duplicadas:")
            colunas_finais = []
            contador = {}

            for col in colunas_limpas:
                if col in duplicadas:
                    if col not in contador:
                        contador[col] = 0
                        colunas_finais.append(col)
                    else:
                        contador[col] += 1
                        novo_nome = f"{col}_dup{contador[col]}"
                        colunas_finais.append(novo_nome)
                        self._log(f"      '{col}' → '{novo_nome}'")
                else:
                    colunas_finais.append(col)

            df.columns = colunas_finais

        # 5. Remover linhas de totais/resultados
        padroes_remover = [
            r'(?i)^total',
            r'(?i)^resultado',
            r'(?i)^soma',
            r'(?i)^subtotal',
            r'(?i)^grand total'
        ]

        linhas_remover = []
        for idx, row in df.iterrows():
            primeira_celula = str(row.iloc[0]).strip().lower()
            for padrao in padroes_remover:
                if re.search(padrao, primeira_celula):
                    linhas_remover.append(idx)
                    break

        if linhas_remover:
            self._log(f"   🗑️  Removendo {len(linhas_remover)} linhas de totais/resultados")
            df = df.drop(index=linhas_remover)

        # 6. Reset index
        df = df.reset_index(drop=True)

        self.df_limpo = df

        self._log(f"\n✅ Limpeza concluída")
        self._log(f"   Registros finais: {len(self.df_limpo):,}")
        self._log(f"   Colunas finais: {len(self.df_limpo.columns)}")

    def _gerar_relatorio(self):
        """
        Gera relatório final de detecção
        """
        self._log("\n" + "="*80, nivel="TITULO")
        self._log("📋 RELATÓRIO FINAL DE DETECÇÃO", nivel="TITULO")
        self._log("="*80, nivel="TITULO")

        self._log(f"\n📁 Arquivo: {self.nome_arquivo}")
        self._log(f"📊 Sheet: {self.sheet_detectada}")
        self._log(f"📍 Cabeçalho: Linha {self.linha_cabecalho + 1} (Excel)")
        self._log(f"📍 Dados: Linha {self.linha_dados_inicio + 1} (Excel)")

        self._log(f"\n📊 Resultado Final:")
        self._log(f"   Registros: {len(self.df_limpo):,}")
        self._log(f"   Colunas: {len(self.df_limpo.columns)}")

        self._log(f"\n📋 Colunas detectadas:")
        for i, col in enumerate(self.df_limpo.columns, 1):
            self._log(f"   {i:2d}. {col}")

    def _log(self, mensagem, nivel="INFO"):
        """
        Sistema de log
        """
        timestamp = datetime.now().strftime("%H:%M:%S")

        if nivel == "TITULO":
            print(mensagem)
        elif nivel == "SECAO":
            print(f"\n{mensagem}")
            print("─" * 80)
        else:
            print(mensagem)

        self.log.append({
            'timestamp': timestamp,
            'nivel': nivel,
            'mensagem': mensagem
        })

    def exportar_log(self, pasta_destino):
        """
        Exporta log para arquivo
        """
        log_df = pd.DataFrame(self.log)
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        arquivo_log = Path(pasta_destino) / f"LOG_DeteccaoAutomatica_{timestamp}.xlsx"

        log_df.to_excel(arquivo_log, index=False)
        print(f"\n💾 Log salvo: {arquivo_log}")

        return arquivo_log


# ═══════════════════════════════════════════════════════════════════
# EXEMPLO DE USO
# ═══════════════════════════════════════════════════════════════════

if __name__ == "__main__":
    # Usar FileManager se disponível
    if 'fm' in dir():
        pasta_entrada = fm.diretorios['dados_entrada']
        pasta_logs = fm.diretorios['logs']
    else:
        pasta_entrada = Path('02_Dados_Entrada')
        pasta_logs = Path('logs')

    # Selecionar arquivo
    arquivos_disponiveis = list(pasta_entrada.glob('*.xls*'))

    print("📁 Arquivos disponíveis:")
    for i, arq in enumerate(arquivos_disponiveis, 1):
        print(f"   {i}. {arq.name}")

    escolha = int(input("\nEscolha um arquivo (número): ")) - 1
    arquivo_selecionado = arquivos_disponiveis[escolha]

    # Processar
    detector = DetectorArquivoDesconhecido(arquivo_selecionado)
    df_resultado = detector.processar()

    # Exportar log
    detector.exportar_log(pasta_logs)

    # Salvar resultado
    arquivo_saida = pasta_entrada.parent / '03_Dados_Processados' / f"Detectado_{arquivo_selecionado.stem}.xlsx"
    df_resultado.to_excel(arquivo_saida, index=False)
    print(f"💾 Resultado salvo: {arquivo_saida}")

📁 Arquivos disponíveis:
   1. AIVI_OPAV_BW_20251015_110105.xlsx
   2. Centros_BR_20251015_110105.xlsx
   3. Cópia de xSAPtemp4687_JAN_25.xls
   4. SAP_YSMM_Limpo_20251015_110105.xlsx


ValueError: invalid literal for int() with base 10: ''